In [1]:
%run "..\general_functions\generalFunctions.ipynb"
%run "..\Landscape slide duplicate\Landscape Replacement Function New.ipynb"

In [2]:
start_time = time.time()

## Data Parameters

In [3]:
client_manuf = ["Bel"]
client_brands = ["Kiri", "La Vache Qui Rit", "Boursin"]

rpvm_comp_brands = []
 
decimals = 2
sign = "After"
currency = '€'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '

categories = ["Total Fromage"]
sectors = ["Soft Cheese", "Aperitif", "Ingredient A Chaud"]
segments = ["Enfant", "Frais A Tartiner", "Salade"]
subsegments= []
subcategories= []

national = True
customareas= ""
areas = ["NATIONAL", "RETAILER"]

regions_RET  = ["Carrefour", "Intermarche"]
channels_RET = []
market_RET = []

regions_CHAN = []
channels_CHAN = []
market_CHAN = []

regions_REG = []
channels_REG = []
market_REG = []

regions_CUST = []
channels_CUST = []
market_CUST = []
 
slides_Period = "P3M"
National = ["National"]

data_source = "DATA SOURCE: Trade Panel/Retailer Data | July 2025" 
 
years = {2023,2024,2025}


ManufOrTopC ="Top Companies"
BrandOrTopB = "Top Brands"
end_date = "2025-08-01"


# subcatg_parent = "Segment"
# subcatg_parent_list = segments
percent = 1000000
percentstr="'000 000"

Sections_WO_CB=True

In [4]:
retMarketLength = 3-((not regions_RET)+(not channels_RET)+(not market_RET))
chanMarketLength = 3-((not regions_CHAN)+(not channels_CHAN)+(not market_CHAN))
regMarketLength = 3-((not regions_REG)+(not channels_REG)+(not market_REG))
custMarketLength = 3-((not regions_CUST)+(not channels_CUST)+(not market_CUST))

In [5]:
NATIONAL_area = National
retailers = regions_RET + channels_RET + market_RET
channels = regions_CHAN + channels_CHAN + market_CHAN
regions = regions_REG + channels_REG + market_REG
cust = regions_CUST + channels_CUST + market_CUST

### Cleaning Data_frames

In [6]:
loaded_data = {}

datasets_path = os.getcwd()+"/Landscape Datasets NewEX/"

print(datasets_path)

datasets = os.listdir(datasets_path)
for d in datasets:
    with open(datasets_path+d, 'rb') as handle:
        globals()[d.split('.')[0]] = pd.read_pickle(handle)

c:\Users\BW4SA\Documents\Slide-Automate\Landscape slide duplicate/Landscape Datasets NewEX/


In [7]:
def process_date(manuf_dfs, dfs_total, datacolumn):
    output = {} 
    
    for k, df in manuf_dfs.items():
        df = df.copy()
        first_col = df.columns[0]
        df_pivot = df.pivot_table(index=first_col, columns=datacolumn, aggfunc='sum')  
        df_pivot.columns = [f"{metric} {year}" for metric, year in df_pivot.columns]  # Preserve space
        df_pivot.reset_index(inplace=True)
        
        for col in df_pivot.columns:
            if 'Sales' in col and not pd.api.types.is_numeric_dtype(df_pivot[col]):
                df_pivot[col] = pd.to_numeric(df_pivot[col], errors='coerce')
        
        numeric_cols = df_pivot.select_dtypes(include=['number']).columns
        years = sorted(df[datacolumn].dropna().unique(), key=list(df[datacolumn]).index)
        metrics = sorted(set(col.rsplit(" ", 1)[0] for col in numeric_cols), 
                         key=lambda x: list(df.columns).index(x) if x in df.columns else float('inf'))
        sorted_columns = [first_col] + [f"{metric} {year}" for year in years for metric in metrics]
        df_pivot = df_pivot[[col for col in sorted_columns if col in df_pivot.columns]]
        df_pivot = df_pivot.loc[~(df_pivot.select_dtypes(include=['number']) == 0).all(axis=1)]
        # df_pivot = df_pivot[~df_pivot[df_pivot.columns[0]].astype(str).str.contains('Grand Total', na=False)]

        numeric_cols = df_pivot.select_dtypes(include=['number']).columns  

        # Merge df_pivot with df_total
        if k in dfs_total:
            df_total = dfs_total[k]
            # Ensure the columns match between df_total and df_pivot
            merge_key = df_pivot.columns[0]  # or explicitly set, e.g., 'Segment'
            df_pivot = pd.merge(df_pivot, df_total, how='left', on=merge_key)

        # Store the result in the output dictionary
        output[k] = df_pivot
    
    return output


In [8]:
def secsegclean(inputdic):
    outputdic = {}
    totaloutputdic = {}

    for s in inputdic.keys():
        t = inputdic[s].copy()

        # Sort by 'Total Value Share' columns if they exist
        if not t.empty:
            sort_columns = [col for col in t.columns if 'Value Share' in col]
            if sort_columns:
                # Place "Grand Total" at the bottom after sorting
                non_total = t[~t[t.columns[0]].astype(str).str.contains('Grand Total', na=False)]
                grand_total = t[t[t.columns[0]].astype(str).str.contains('Grand Total', na=False)]
                non_total_sorted = non_total.sort_values(sort_columns, ascending=False)
                # t = pd.concat([non_total_sorted,grand_total], ignore_index=True)
                
                if not non_total_sorted.empty:
                    outputdic[s] = non_total_sorted

                # Also store only the Grand Total row(s) separately if needed
                tot = t[t[t.columns[0]].astype(str).str.contains('Grand Total', na=False)]
                if not tot.empty:
                    totaloutputdic[s] = tot

    return outputdic, totaloutputdic


In [9]:
sectors_dfs=process_date(Sector_dfs,Sector_total_dfs,"Year")
segments_dfs=process_date(Segment_dfs,Segment_total_dfs,"Year")
subsegments_dfs=process_date(SubSegment_dfs,SubSegment_total_dfs,"Year")
subcategories_dfs=process_date(SubCategory_dfs,SubCategory_total_dfs,"Year")


In [10]:
if len(sectors)!=0:
    modified_sectors_dfs_new = {}
    sectors_totals_new= {}
    modified_sectors_dfs_new, sectors_totals_new = secsegclean(sectors_dfs)
    #P12M
    globals()[f"modified_sectors_{slides_Period}_new"] = {}
    dya_sectors_new = {}
    globals()[f"modified_sectors_{slides_Period}_new"], dya_sectors_new = secsegclean(globals()[f"Sector_{slides_Period}_dfs"])
    
    #Client
    modified_sectors_clients_new = {}
    modified_sectors_clients_new,_ = secsegclean(Sector_client_dfs)

if len(segments)!=0:
    modified_segment_dfs_new = {}
    segment_totals_new= {}
    modified_segment_dfs_new, segment_totals_new = secsegclean(segments_dfs)
    #P12M
    globals()[f"modified_segment_{slides_Period}_new"] = {}
    dya_segment_new = {}
    globals()[f"modified_segment_{slides_Period}_new"], dya_segment_new = secsegclean(globals()[f"Segment_{slides_Period}_dfs"])
    #Client
    modified_segment_clients_new = {}
    modified_segment_clients_new,_ = secsegclean(Segment_client_dfs)


if len(subsegments)!=0:
    modified_subsegment_dfs_new = {}
    subsegment_totals_new= {}
    modified_subsegment_dfs_new, subsegment_totals_new = secsegclean(subsegments_dfs)
    #P12M
  
    globals()[f"modified_subsegment_{slides_Period}_new"] = {}
    dya_subsegment_new = {}
    globals()[f"modified_subsegment_{slides_Period}_new"], dya_subsegment_new = secsegclean(globals()[f"SubSegment_{slides_Period}_dfs"])
    #Client
    modified_subsegment_clients_new = {}
    modified_subsegment_clients_new,_ = secsegclean(SubSegment_client_dfs)


if len(subcategories)!=0:
    modified_subcategories_dfs_new = {}
    subcategories_totals_new= {}
    modified_subcategories_dfs_new, subcategories_totals_new = secsegclean(subcategories_dfs)
    #P12M
    globals()[f"modified_subcategories_{slides_Period}_new"] = {}
    dya_subcategories_new = {}
    globals()[f"modified_subcategories_{slides_Period}_new"], dya_subcategories_new = secsegclean(globals()[f"SubCategory_{slides_Period}_dfs"])
    #Client
    modified_subcategories_clients_new = {}
    modified_subcategories_clients_new,_ = secsegclean(SubCategory_client_dfs)


In [11]:
def rename_grand_total(dataframe_dict):
    for k in dataframe_dict.keys():
        main_category = k.split("|")[0].strip()
        first_col = dataframe_dict[k].columns[0]  # Dynamic column name
        dataframe_dict[k].loc[dataframe_dict[k][first_col] == 'Grand Total', first_col] = main_category
        df = dataframe_dict[k]
        if len(df) > 1:  # Only if there's more than one row
            # Get the last row
            last_row = df.iloc[-1:].copy()
            # Get all rows except the last one
            df_without_last = df.iloc[:-1].copy()
            # Concatenate with last row first
            dataframe_dict[k] = pd.concat([last_row, df_without_last], ignore_index=True)

# Apply to all dictionaries
if len(sectors)!=0:
    rename_grand_total(modified_sectors_clients_new)
    rename_grand_total(globals()[f"modified_sectors_{slides_Period}_new"])
    
if len(segments)!=0:    
    rename_grand_total(modified_segment_clients_new)
    rename_grand_total(globals()[f"modified_segment_{slides_Period}_new"])
    
if len(subsegments)!=0:
    rename_grand_total(modified_subsegment_clients_new)
    rename_grand_total(globals()[f"modified_subsegment_{slides_Period}_new"])
    
if len(subcategories)!=0:
    rename_grand_total(modified_subcategories_clients_new)
    rename_grand_total(globals()[f"modified_subcategories_{slides_Period}_new"])

In [12]:
def national_rows(data_dict):
    filtered_dict = {}
    for key, df in data_dict.items():
        # Create a copy to avoid SettingWithCopyWarning
        df = df.copy()
        df['Area'] = df['Area'].replace('NATIONAL', 'National')
        
        # Filter for national rows (case-insensitive)
        national_df = df[df['Area'].str.lower() == 'national']
        
        if not national_df.empty:
            filtered_dict[key] = national_df.reset_index(drop=True)
    return filtered_dict
modified_national = national_rows(modified_national)
modified_national_client = national_rows(modified_national_client)


In [13]:
def addOther(dic,col=f"{BrandOrTopB}",TorB="Top"):
    for key, df in dic.items():
        if col in df.columns:
            others_row = df[df[col].str.contains('Others', case=False, na=False)]
            rows_wo_others = df[~df[col].str.contains('Others', case=False, na=False)]
            if not others_row.empty:
                if TorB=="Top":
                # Re-order: "Others" first, then the rest of the rows
                    dic[key] = pd.concat([others_row, rows_wo_others])    
                else:
                    dic[key] = pd.concat([ rows_wo_others,others_row])    
    return dic                    

In [14]:
manuf_dfs_new=process_date(manuf_dfs,manuf_total_dfs,"Year")
globals()[f"manuf_{slides_Period}_dfs_new"]=process_date(globals()[f"manuf_{slides_Period}_dfs"],globals()[f"manuf_{slides_Period}_total_dfs"],"Time Period")
brands_dfs_new=process_date(brand_dfs,brand_total_dfs,"Year")


In [15]:
modified_manuf_dfs_new = {}
modified_manuf_totals_new= {}
# modified_manuf_P12M_new = {}
# modified_manuf_P12M_regions_new = {}
globals()[f"modified_manuf_{slides_Period}_new"] = {}
globals()[f"modified_manuf_{slides_Period}_regions_new"] = {}

for s in manuf_dfs_new.keys():
    t = manuf_dfs_new[s].copy()
    grand_total_values = []
    t.columns = t.columns.str.strip()            
    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    numeric_cols = mod.select_dtypes(include=['number']).columns
    mod = mod.loc[~(mod[numeric_cols] == 0).all(axis=1)]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    if not mod.empty:
        modified_manuf_dfs_new[s] = mod

    if not tot.empty:
        modified_manuf_totals_new[s] = tot
    

df_toDrop =[key for key in modified_manuf_dfs_new.keys() if (modified_manuf_dfs_new[key].shape[1]>2) and (modified_manuf_dfs_new[key].shape[1]<4)]

for key in df_toDrop:
    del modified_manuf_dfs_new[key]

emptyDf=[key for key in modified_manuf_dfs_new.keys() if modified_manuf_dfs_new[key].shape[1]<4]

for dic in [modified_manuf_dfs_new]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            dic[df]=dic[df].drop(columns=[''], errors = 'ignore')#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0
modified_manuf_dfs_new = dfSort(modified_manuf_dfs_new, client_manuf,f'{ManufOrTopC}',10, 'Total Value Share')
addOther(modified_manuf_dfs_new,col=f'{ManufOrTopC}',TorB="Bottom")

manuf_per = globals()[f"manuf_{slides_Period}_dfs_new"]
 
for s in manuf_per.keys():
    t = manuf_per[s].copy()
    grand_total_values = []
    t.columns = t.columns.str.strip()            

    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    if not mod.empty:
        globals()[f"modified_manuf_{slides_Period}_new"][s] = mod

df_toDrop =[key for key in globals()[f"modified_manuf_{slides_Period}_new"].keys() if (globals()[f"modified_manuf_{slides_Period}_new"][key].shape[1]>2) and (globals()[f"modified_manuf_{slides_Period}_new"][key].shape[1]<4)]
for key in df_toDrop:
    del globals()[f"modified_manuf_{slides_Period}_new"][key]


emptyDf=[key for key in globals()[f"modified_manuf_{slides_Period}_new"].keys() if globals()[f"modified_manuf_{slides_Period}_new"][key].shape[1]<4]
for dic in [globals()[f"modified_manuf_{slides_Period}_new"]]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            dic[df]=dic[df].drop(columns=[''])#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0

globals()[f"modified_manuf_{slides_Period}_new"] = dfSort(globals()[f"modified_manuf_{slides_Period}_new"], client_manuf,f'{ManufOrTopC}',10, 'Value Share')
addOther(globals()[f"modified_manuf_{slides_Period}_new"],col=f"{ManufOrTopC}",TorB="Bottom")



{'Total Fromage | NATIONAL':   Top Companies  Volume Sales P3M  Value Sales P3M  Value Share P3M  \
 0           Mdd        62873789.0      623932113.0         0.405691   
 1      Savencia        18769117.0      230846945.0         0.150100   
 2      Lactalis        18248235.0      221058561.0         0.143736   
 2           Bel        11222736.0      161364371.0         0.104922   
 4        Eurial         4365301.0       54497273.0         0.035435   
 5  Casa Azzurra         2835434.0       43075846.0         0.028009   
 6         Laita         2423370.0       27506509.0         0.017885   
 0        Others        10595624.0      175667847.0         0.114222   
 
    Volume Share P3M  Share DYA P3M  Av Price/KG P3M  Volume Sales LY  \
 0          0.478733      -0.079162          9.92360      495514275.0   
 1          0.142912       0.042900         12.29930       94937889.0   
 2          0.138946      -0.009707         12.11400      119297444.0   
 2          0.085452       0.0

In [16]:
modified_brands_share_new = {}
modified_brands_totals_new = {}
globals()[f"modified_brands_{slides_Period}_new"] = {}
modified_brands_evolution_new = {}
modified_manuf_evolution_new = {}


for s in brands_dfs_new.keys():
    t = brands_dfs_new[s].copy()
    t.columns = t.columns.str.strip()            
    grand_total_values = []
    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    if not mod.empty:
        modified_brands_share_new[s] = mod
    if not tot.empty:
        modified_brands_totals_new[s] = tot


df_toDrop =[key for key in modified_brands_share_new.keys() if (modified_brands_share_new[key].shape[1]>2) and (modified_brands_share_new[key].shape[1]<4)]
for key in df_toDrop:
    del modified_brands_share_new[key]


emptyDf=[key for key in modified_brands_share_new.keys() if modified_brands_share_new[key].shape[1]<4]
for dic in [modified_brands_share_new]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            dic[df]=dic[df].drop(columns=[''])#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0
p = 10 if len(client_brands) < 10 else len(client_brands) + 1

modified_brands_share_new = dfSort(modified_brands_share_new, client_brands,f'{BrandOrTopB}',p, 'Total Value Share')
for key in modified_brands_share_new.keys():
    modified_brands_share_new[key] = modified_brands_share_new[key][modified_brands_share_new[key]['Total Value Share'] != 0]
# Move rows containing "Others" in 'Top Brands' column to the first row
addOther(modified_brands_share_new,col=f'{BrandOrTopB}',TorB="Bottom")

brand_per=globals()[f"brand_{slides_Period}_dfs"]
for s in brand_per.keys():
    t = brand_per[s].copy()
    grand_total_values = []
    t.columns = t.columns.str.strip()            
    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    globals()[f"modified_brands_{slides_Period}_new"][s] = mod

df_toDrop =[key for key in globals()[f"modified_brands_{slides_Period}_new"].keys() if (globals()[f"modified_brands_{slides_Period}_new"][key].shape[1]>2) and (globals()[f"modified_brands_{slides_Period}_new"][key].shape[1]<4)]
for key in df_toDrop:
    del globals()[f"modified_brands_{slides_Period}_new"][key]

emptyDf=[key for key in globals()[f"modified_brands_{slides_Period}_new"].keys() if globals()[f"modified_brands_{slides_Period}_new"][key].shape[1]<4]
for dic in [globals()[f"modified_brands_{slides_Period}_new"]]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            dic[df]=dic[df].drop(columns=[''])#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0

globals()[f"modified_brands_{slides_Period}_new"] = dfSort(globals()[f"modified_brands_{slides_Period}_new"], client_brands,f'{BrandOrTopB}',10, 'Value Share')
addOther(globals()[f"modified_brands_{slides_Period}_new"],col=f'{BrandOrTopB}',TorB="Bottom")

    

{'Total Fromage | NATIONAL':            Top Brands  Volume Sales  Value Sales  Value Share  Volume Share  \
 0       Private Label    62873789.0  623932113.0     0.405691      0.478733   
 2           President     7719733.0   74730598.0     0.048591      0.058780   
 3             Galbani     4599888.0   60716546.0     0.039479      0.035024   
 4            St Moret     4201349.0   54596188.0     0.035499      0.031990   
 5             Soignon     4020699.0   49577769.0     0.032236      0.030614   
 6   Caprice Des Dieux     3507982.0   47071227.0     0.030606      0.026710   
 11   La Vache Qui Rit     3380849.0   35786185.0     0.023269      0.025742   
 48               Kiri     2242846.0   29293535.0     0.019047      0.017077   
 3             Boursin     1482770.0   26066303.0     0.016949      0.011290   
 0              Others    37303701.0  536179001.0     0.348632      0.284038   
 
     Share DYA  Av Price/KG  IYA Price/KG  
 0   -0.079162     9.923600      0.981000  
 2

In [17]:
def evolution_cleaning(dic,colname=f"{BrandOrTopB}"):
    modified_evolution_new={}
    modified_evolution_sorted_new = {}
    for s in dic.keys():
        t = dic[s].copy()
        t.columns = t.columns.str.strip()            
        t = t.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
        t = t.applymap(lambda x: 0 if pd.isna(x) else x)
        mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
        mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
        tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
        if not mod.empty:
            modified_evolution_new[s] = mod
    for k in modified_evolution_new.keys():
        x = modified_evolution_new[k].copy()
        x[colname] = np.where(x[colname]==0, x['Year'], x[colname])
        x[colname] = x[colname].str.replace('2022 ','').str.replace('2021 ','').str.replace('2023 ',"").str.replace('2020 ','').str.replace('2024 ','').str.replace('2025 ','')
        x = x[x[colname] != 'Others']
        x = x[x[colname] != 'All Others']
        x = x[x['Price Evolution'] != 0]
        brand_sales = x.groupby(colname)['Value Sales'].sum().sort_values(ascending=False).head(5)
        if colname==f"{BrandOrTopB}":
            final = x[(x[colname].isin(pd.Series(brand_sales.index))) | (x[colname].isin(client_brands))]
        else:
            final = x[(x[colname].isin(pd.Series(brand_sales.index))) | (x[colname].isin(client_manuf))]
        final[['Price Evolution', 'Share Evolution']] = round(final[['Price Evolution', 'Share Evolution']].replace(np.nan,0).astype(float)*100).astype(int)
        final = final.drop_duplicates(subset = ["Year", colname])
        final = final.sort_values('Year')  
        final["Year"]=final["Year"].astype(int)      
        modified_evolution_sorted_new[k] = final
    keys_to_remove = []
    for key, df in modified_evolution_sorted_new.items():
        if df.empty:  # Remove the () from df.empty
            keys_to_remove.append(key)
 
    for key in keys_to_remove:
        del modified_evolution_sorted_new[key]
 
    return modified_evolution_sorted_new

In [18]:
def add_tot(dic, totdic):
    result_dic = {}  # Store results for each key
    for k in dic.keys():
        df = dic[k].copy()
        dt = totdic[k].copy()
        final_rows = []
        for year, group in df.groupby("Year"):
            final_rows.append(group)  # Append the original group
            total_row = dt[dt["Year"] == year]
            if not total_row.empty:
                total_row = total_row.copy()
                total_row["Year"] = f"{year} Total"  # Set as "Total <Year>"
                final_rows.append(total_row)  # Append the modified total row
        final_df = pd.concat(final_rows, ignore_index=True)
        result_dic[k] = final_df  # Store the result for each key
    return result_dic  # Return a dictionary of processed DataFrames


In [19]:
modified_brands_evolution_sorted=evolution_cleaning(brands_evolution,colname=f'{BrandOrTopB}')
modified_manuf_evolution_sorted=evolution_cleaning(manuf_evolution,colname=f'{ManufOrTopC}')
modified_brands_evolution_sorted_new=add_tot(modified_brands_evolution_sorted,brands_evolution_total)
modified_manuf_evolution_sorted_new=add_tot(modified_manuf_evolution_sorted,brands_evolution_total)


In [20]:
def grandtot(dic,marketlis):
    cleaned_dic = {}  # Store cleaned DataFrames without "Grand Total"
    total_dic = {}  # Store DataFrames with only "Grand Total"
    for k in dic.keys():
        df = dic[k].copy()
        total_df = df[df[df.columns[0]] == "Grand Total"]
        cleaned_df = df[df[df.columns[0]] != "Grand Total"]
        cleaned_df = cleaned_df[cleaned_df[cleaned_df.columns[0]].isin(marketlis)]
        if not cleaned_df.empty:
            cleaned_dic[k] = cleaned_df
        if not total_df.empty:
            total_dic[k] = total_df

    return cleaned_dic, total_dic


In [21]:
modified_retailer_regions = {}
modified_retailer_channels = {}
modified_retailer_markets = {}
 
modified_channels_regions = {}
modified_channels_channels = {}
modified_channels_markets = {}
 
modified_regions_regions = {}
modified_regions_channels = {}
modified_regions_markets = {}
 
modified_cust_regions = {}
modified_cust_channels = {}
modified_cust_markets = {}
 
dya_retailer_reg={}
dya_retailer_channel={}
dya_retailer_mkt = {}
 
dya_channel_reg={}
dya_channel_channel={}
dya_channel_mkt = {}
 
dya_region_reg={}
dya_region_channel={}
dya_region_mkt = {}
 
dya_cust_reg={}
dya_cust_channel={}
dya_cust_mkt = {}
 
# ********** Retailer *********
if len(retailers) != 0:
    modified_retailer_regions,dya_retailer_reg = grandtot(globals().get("modified_retailer_Region", {}) or {},regions_RET)
    modified_retailer_channels,dya_retailer_channel =grandtot( globals().get("modified_retailer_Channel", {}) or {},channels_RET)
    modified_retailer_markets,dya_retailer_mkt = grandtot(globals().get("modified_retailer_Market", {}) or {},market_RET)
 
# ********* Channels **********
if len(channels) != 0:
    modified_channels_regions,dya_channel_reg = grandtot(globals().get("modified_channels_Region", {}) or {},regions_CHAN)
    modified_channels_channels,dya_retailer_channel = grandtot(globals().get("modified_channels_Channel", {}) or {},channels_CHAN)
    modified_channels_markets,dya_retailer_mkt = grandtot(globals().get("modified_channels_Market", {}) or {},market_CHAN)
 
 
if len(regions) != 0:
    modified_regions_regions,dya_region_reg = grandtot(globals().get("modified_regions_Region", {}) or {},regions_REG)
    modified_regions_channels,dya_region_channel = grandtot(globals().get("modified_regions_Channel", {}) or {},channels_REG)
    modified_regions_markets,dya_region_mkt = grandtot(globals().get("modified_regions_Market", {}) or {},market_REG)
 
# ******** POS ********
if len(cust) != 0:
    modified_cust_regions,dya_cust_reg = grandtot(globals().get("modified_cust_Region", {}) or {},regions_CUST)
    modified_cust_channels,dya_cust_channel = grandtot(globals().get("modified_cust_Channel", {}) or {},channels_CUST)
    modified_cust_markets,dya_cust_mkt= grandtot(globals().get("modified_cust_Market", {}) or {},market_CUST)

In [22]:
#### Client
modified_retailer_regions_client = {}
modified_retailer_channels_client = {}
modified_retailer_markets_client = {}

modified_channels_regions_client = {}
modified_channels_channels_client = {}
modified_channels_markets_client = {}

modified_regions_regions_client = {}
modified_regions_channels_client = {}
modified_regions_markets_client = {}

modified_cust_regions_client = {}
modified_cust_channels_client = {}
modified_cust_markets_client = {}


dya_retailer_reg_client={}
dya_retailer_channel_client={}
dya_retailer_mkt_client = {}
 
dya_channel_reg_client={}
dya_channel_channel_client={}
dya_channel_mkt_client= {}
 
dya_region_reg_client={}
dya_region_channel_client={}
dya_region_mkt_client= {}
 
dya_cust_reg_client={}
dya_cust_channel_client={}
dya_cust_mkt_client= {}

#********** Retailer *********
if len(retailers) != 0:
    modified_retailer_regions_client,dya_retailer_reg_client = grandtot(globals().get("modified_retailer_Region_client", {}) or {},regions_RET)
    modified_retailer_channels_client,dya_retailer_channel_client = grandtot(globals().get("modified_retailer_Channel_client", {}) or {},channels_RET)
    modified_retailer_markets_client,dya_retailer_mkt_client = grandtot(globals().get("modified_retailer_Market_client", {}) or {},market_RET)

# ********* Channels **********
if len(channels) != 0:
    modified_channels_regions_client,dya_channel_reg_client = grandtot(globals().get("modified_channels_Region_client", {}) or {},regions_CHAN)
    modified_channels_channels_client,dya_channel_channel_client = grandtot(globals().get("modified_channels_Channel_client", {}) or {},channels_CHAN)
    modified_channels_markets_client,dya_channel_mkt_client=grandtot(globals().get("modified_channels_Market_client", {}) or {},market_CHAN)

# ******** Regions ********
if len(regions) != 0: 
    modified_regions_regions_client,dya_region_reg_client = grandtot(globals().get("modified_regions_Region_client", {}) or {},regions_REG)
    modified_regions_channels_client,dya_region_channel_client = grandtot(globals().get("modified_regions_Channel_client", {}) or {},channels_REG)
    modified_regions_markets_client,dya_region_mkt_client = grandtot(globals().get("modified_regions_Market_client", {}) or {},market_REG)

# ******** POS ********
if len(cust) != 0: 
    modified_cust_regions_client,dya_cust_reg_client = grandtot(globals().get("modified_cust_Region_client", {}) or {},regions_CUST)
    modified_cust_channels_client,dya_cust_channel_client= grandtot(globals().get("modified_cust_Channel_client", {}) or {},channels_CUST)
    modified_cust_markets_client,dya_cust_mkt_client = grandtot(globals().get("modified_cust_Market_client", {}) or {},market_CUST)



In [23]:
modified_calendar_new = {}
for table in calendar_dfs.keys():
    modified_tables = []
    cleantable = calendar_dfs[table].copy()
    cleantable =cleantable[cleantable['MonthYear']!='Grand Total']
    cleantable['time'] = pd.to_datetime(cleantable['MonthYear'], format='%b-%y')
    cleantable = cleantable.nlargest(25, 'time')
    cleantable = cleantable.sort_values('time')
    cleantable['Av Price/KG'] = cleantable['Av Price/KG'].replace(np.nan, 0)
    cleantable['Av Price/KG'] =round(cleantable['Av Price/KG'].astype(float),decimals)
    modified_calendar_new[table] = cleantable


keys_to_remove = []
for key, df in modified_calendar_new.items():
    if df.empty:  # Remove the () from df.empty
        keys_to_remove.append(key)

for key in keys_to_remove:
    del modified_calendar_new[key]


In [24]:
modified_categories_dfs_new=[]
for key in categories_overview_dfs.keys():
    try:
        dfCat=categories_overview_dfs[key].copy()
        dfCat[['Cat Volume Sales/M','Cat Value Sales/M']]=round(dfCat[['Volume Sales','Value Sales']].astype(float)/10**6, 2)
        dfCat[['Cat Volume Sales Change YA','Cat Value Sales Change YA']]=round((dfCat[['Volume Sales IYA','Value Sales IYA']].astype(float)*100)-100,2)
        dfCat['Cat IYA Price/KG']=round((dfCat['IYA Price/KG'].astype(float)*100),decimals)
        dfCat['flag']=1

        dfMamu=categories_overview_manuf_dfs[key].copy()
        
        dfMamu[['Manu Volume Sales/M','Manu Value Sales/M']]=round(dfMamu[['Volume Sales','Value Sales']].astype(float)/10**6, 2)
        dfMamu[['Manu Volume Sales Change YA','Manu Value Sales Change YA']]=round((dfMamu[['Volume Sales IYA','Value Sales IYA']].astype(float)*100)-100,2)
        dfMamu['Manu IYA Price/KG']=round((dfMamu['IYA Price/KG'].astype(float)*100),2)
        dfMamu['Impact Growth'] = dfMamu['Manu Value Sales Change YA'] - dfMamu['Manu Volume Sales Change YA']
        dfMamu['flag']=1

        table = pd.merge(dfCat[['flag','Cat Volume Sales/M','Cat Value Sales/M','Cat Volume Sales Change YA','Cat Value Sales Change YA','Cat IYA Price/KG']],dfMamu[['flag','Manu Volume Sales/M','Manu Value Sales/M','Manu Volume Sales Change YA','Manu Value Sales Change YA','Manu IYA Price/KG','Impact Growth']]).drop(columns='flag')
        table = table[['Cat Value Sales/M','Cat Value Sales Change YA','Cat IYA Price/KG', 'Manu Value Sales/M','Manu Value Sales Change YA','Manu IYA Price/KG','Cat Volume Sales/M','Cat Volume Sales Change YA','Manu Volume Sales/M','Manu Volume Sales Change YA','Impact Growth']]
        table['Price Impact'] = str(int(round(table['Manu Value Sales Change YA']))-int(round(table['Manu Volume Sales Change YA'])))+"pts"
        
        table['Cat Value Sales Change YA']=str(int(round(table['Cat Value Sales Change YA'])))+'%'
        table['Cat Volume Sales Change YA']=str(int(round(table['Cat Volume Sales Change YA'])))+'%'
        table['Manu Value Sales Change YA']=str(int(round(table['Manu Value Sales Change YA'])))+'%'
        table['Manu Volume Sales Change YA']=str(int(round(table['Manu Volume Sales Change YA'])))+'%'
        table['Cat IYA Price/KG']=str(int(round(table['Cat IYA Price/KG'])))#+'%'
        table['Manu IYA Price/KG']=str(int(round(table['Manu IYA Price/KG'])))#+'%'
        lis=['Cat Value Sales/M','Manu Value Sales/M','Cat Volume Sales/M','Manu Volume Sales/M','Impact Growth']
        for col in lis:
            if round(table[col][0],1)==int(round(table[col][0],1)):
                table[col]=int(round(table[col]))
            else:
                table[col]=round(table[col],1)
        

        dfVal = categories_values_dfs[key].copy()
        first_col = dfVal.columns[0]
        dfValnogr = dfVal[dfVal[first_col] != "Grand Total"]
        grand_total = dfVal[dfVal[first_col] == "Grand Total"]
        ordered_rows = []
        for month in dfValnogr[first_col].unique():
            group = dfValnogr[dfValnogr[first_col] == month]
            total_row = group.sum(numeric_only=True)
            total_row[first_col] = f"{month} Total"  # Append "Total" to the group name
            total_row = pd.DataFrame([total_row])  # Convert to DataFrame
            ordered_rows.append(group)  # Add the group rows
            ordered_rows.append(total_row)  # Add the total row
        df_with_totals = pd.concat(ordered_rows, ignore_index=True)
        df_final = pd.concat([df_with_totals, grand_total], ignore_index=True)
        modified_categories_dfs_new.append([table, df_final, key])

    except Exception as e:
        print(f"Error with key '{key}': {e}")
    

In [25]:
sharGrowthDf_sec={}
sharGrowthDf_seg={}
sharGrowthDf_subseg={}
sharGrowthDf_subcat={}
shareGrowthDf_retailers_REG={}
shareGrowthDf_retailers_CHAN={}
shareGrowthDf_retailers_MAR={}
shareGrowthDf_channels_REG={}
shareGrowthDf_channels_CHAN={}
shareGrowthDf_channels_MAR={}
shareGrowthDf_regions_REG={}
shareGrowthDf_regions_CHAN={}
shareGrowthDf_regions_MAR={}
shareGrowthDf_custom_REG={}
shareGrowthDf_custom_CHAN={}
shareGrowthDf_custom_MAR={}

for key in categories + sectors + segments + subsegments +subcategories:
    
    shareGrowthDf_retailers_REG[key]=[k.split(' | ')[1] for k in globals()[f"modified_manuf_{slides_Period}_new"].keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in regions_RET ] 
    shareGrowthDf_retailers_CHAN[key]=[k.split(' | ')[1] for k in globals()[f"modified_manuf_{slides_Period}_new"].keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in channels_RET ]
    shareGrowthDf_retailers_MAR[key]=[k.split(' | ')[1] for k in globals()[f"modified_manuf_{slides_Period}_new"].keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in market_RET ] 
    
    shareGrowthDf_channels_REG[key]=[k.split(' | ')[1] for k in globals()[f"modified_manuf_{slides_Period}_new"].keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in regions_CHAN ] 
    shareGrowthDf_channels_CHAN[key]=[k.split(' | ')[1] for k in globals()[f"modified_manuf_{slides_Period}_new"].keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in channels_CHAN ]
    shareGrowthDf_channels_MAR[key]=[k.split(' | ')[1] for k in globals()[f"modified_manuf_{slides_Period}_new"].keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in market_CHAN ]

    shareGrowthDf_regions_REG[key]=[k.split(' | ')[1] for k in globals()[f"modified_manuf_{slides_Period}_new"].keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in regions_REG ] 
    shareGrowthDf_regions_CHAN[key]=[k.split(' | ')[1] for k in globals()[f"modified_manuf_{slides_Period}_new"].keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in channels_REG ]
    shareGrowthDf_regions_MAR[key]=[k.split(' | ')[1] for k in globals()[f"modified_manuf_{slides_Period}_new"].keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in market_REG ]

    shareGrowthDf_custom_REG[key]=[k.split(' | ')[1] for k in globals()[f"modified_manuf_{slides_Period}_new"].keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in regions_CUST ] 
    shareGrowthDf_custom_CHAN[key]=[k.split(' | ')[1] for k in globals()[f"modified_manuf_{slides_Period}_new"].keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in channels_CUST ]
    shareGrowthDf_custom_MAR[key]=[k.split(' | ')[1] for k in globals()[f"modified_manuf_{slides_Period}_new"].keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in market_CUST ]

    if len(shareGrowthDf_retailers_REG[key])==0:
        shareGrowthDf_retailers_REG.pop(key,None)    
    if len(shareGrowthDf_retailers_CHAN[key])==0:
        shareGrowthDf_retailers_CHAN.pop(key,None)
    if len(shareGrowthDf_retailers_MAR[key])==0:
        shareGrowthDf_retailers_MAR.pop(key,None)
    
    if len(shareGrowthDf_channels_REG[key])==0:
        shareGrowthDf_channels_REG.pop(key,None)    
    if len(shareGrowthDf_channels_CHAN[key])==0:
        shareGrowthDf_channels_CHAN.pop(key,None)
    if len(shareGrowthDf_channels_MAR[key])==0:
        shareGrowthDf_channels_MAR.pop(key,None)

    if len(shareGrowthDf_regions_REG[key])==0:
        shareGrowthDf_regions_REG.pop(key,None)    
    if len(shareGrowthDf_regions_CHAN[key])==0:
        shareGrowthDf_regions_CHAN.pop(key,None)
    if len(shareGrowthDf_regions_MAR[key])==0:
        shareGrowthDf_regions_MAR.pop(key,None)

    if len(shareGrowthDf_custom_REG[key])==0:
        shareGrowthDf_custom_REG.pop(key,None)    
    if len(shareGrowthDf_custom_CHAN[key])==0:
        shareGrowthDf_custom_CHAN.pop(key,None)
    if len(shareGrowthDf_custom_MAR[key])==0:
        shareGrowthDf_custom_MAR.pop(key,None)
    

for key in ["NATIONAL"] + regions_RET + channels_RET+market_RET +  regions_CHAN +channels_CHAN+ market_CHAN+regions_REG+channels_REG+market_REG+regions_CUST+channels_CUST+ market_CUST:
    sharGrowthDf_sec[key]=[k.split(' | ')[0] for k in globals()[f"modified_manuf_{slides_Period}_new"].keys() if key == k.split(' | ')[1] and k.split(' | ')[0] in sectors ]
    sharGrowthDf_seg[key]=[k.split(' | ')[0] for k in globals()[f"modified_manuf_{slides_Period}_new"].keys() if key == k.split(' | ')[1] and k.split(' | ')[0] in segments]
    sharGrowthDf_subseg[key]=[k.split(' | ')[0] for k in globals()[f"modified_manuf_{slides_Period}_new"].keys() if key == k.split(' | ')[1] and k.split(' | ')[0] in subsegments]
    sharGrowthDf_subcat[key]=[k.split(' | ')[0] for k in globals()[f"modified_manuf_{slides_Period}_new"].keys() if key == k.split(' | ')[1] and k.split(' | ')[0] in subcategories]
    
    if len(sharGrowthDf_sec[key])==0:
        sharGrowthDf_sec.pop(key,None)
        
    if len(sharGrowthDf_seg[key])==0:
        sharGrowthDf_seg.pop(key,None)
    
    if len(sharGrowthDf_subseg[key])==0:
        sharGrowthDf_subseg.pop(key,None)
    
    if len(sharGrowthDf_subcat[key])==0:
        sharGrowthDf_subcat.pop(key,None)

### Create Market Growth Market List for duplication and replacement

In [26]:
markets_name = ['regions_RET', 'channels_RET','market_RET','regions_CHAN' ,'channels_CHAN', 'market_CHAN' ,'regions_REG' ,'channels_REG', 'market_REG','regions_CUST' ,'channels_CUST', 'market_CUST']
markets_val = [regions_RET, channels_RET, market_RET, regions_CHAN, channels_CHAN, market_CHAN, regions_REG, channels_REG, market_REG, regions_CUST, channels_CUST, market_CUST]
markets_dfs = [modified_retailer_regions, modified_retailer_channels,modified_retailer_markets, modified_channels_regions, modified_channels_channels, modified_channels_markets,modified_regions_regions ,modified_regions_channels,  modified_regions_markets,modified_cust_regions ,modified_cust_channels, modified_cust_markets]
market_total=[dya_retailer_reg,dya_retailer_channel,dya_retailer_mkt,dya_channel_reg,dya_channel_channel,dya_channel_mkt,dya_region_reg,dya_region_channel,dya_region_mkt,dya_cust_reg,dya_cust_channel,dya_cust_mkt]
slide_by = ['Region','Channel','Market']*4
slide_for = [*['Retailer']*3,*['Channel']*3,*['Region']*3,*[customareas]*3]
market_list_Growth={}
duplication_num = []
section_name_Growth = []
for idx,val in enumerate(markets_val):
    if len(val)>0:
        market_list_Growth[markets_name[idx]]=[val,markets_dfs[idx],market_total[idx//3],slide_by[idx],slide_for[idx]]
        duplication_num.append(len(markets_dfs[idx].keys()))
        section_name_Growth.append('Market Growth By '+slide_for[idx]+' For '+slide_by[idx])
section_number = len(market_list_Growth.keys())


#####  Create Value_AvgPrice Market List for duplication and replacement

In [27]:
if national:
    def merge_regional_national(dfs_list, modified_national, modified_national_client):
        # Split into standard (first 9) and client (last 9) datasets
        standard_dfs = dfs_list[:9]
        client_dfs = dfs_list[9:]
        
        # Process standard datasets with modified_national
        processed_standard = []
        for df_dict in standard_dfs:
            merged_dict = {}
            for key, df_regional in df_dict.items():
                if df_regional.empty:
                    merged_dict[key] = df_regional
                    continue
                    
                df_national = modified_national.get(key, pd.DataFrame())
                if df_national.empty:
                    merged_dict[key] = df_regional
                    continue
                    
                # Align first column names
                region_col = df_regional.columns[0]
                df_national_aligned = df_national.rename(
                    columns={df_national.columns[0]: region_col}
                )
                
                # Concatenate with national first
                merged_df = pd.concat([df_national_aligned, df_regional], ignore_index=True)
                merged_df.columns = merged_df.columns.str.strip()
                merged_dict[key] = merged_df
                
            processed_standard.append(merged_dict)
        
        # Process client datasets with modified_national_client
        processed_client = []
        for df_dict in client_dfs:
            merged_dict = {}
            for key, df_regional in df_dict.items():
                if df_regional.empty:
                    merged_dict[key] = df_regional
                    continue
                    
                df_national = modified_national_client.get(key, pd.DataFrame())
                if df_national.empty:
                    merged_dict[key] = df_regional
                    continue
                    
                # Align first column names
                region_col = df_regional.columns[0]
                df_national_aligned = df_national.rename(
                    columns={df_national.columns[0]: region_col}
                )
                
                # Concatenate with national first
                merged_df = pd.concat([df_national_aligned, df_regional], ignore_index=True)
                merged_df.columns = merged_df.columns.str.strip()
                merged_dict[key] = merged_df
                
            processed_client.append(merged_dict)
        
        # Combine both processed lists
        all_results = processed_standard + processed_client
        
        # Return as tuple of 18 dataframes
        return tuple(all_results)

    # Usage:
    dfs = [
        modified_retailer_regions, modified_retailer_channels, modified_retailer_markets,
        modified_channels_regions, modified_channels_channels, modified_channels_markets,
        modified_regions_regions, modified_regions_channels, modified_regions_markets,
        modified_cust_regions, modified_cust_channels, modified_cust_markets,
        modified_retailer_regions_client, modified_retailer_channels_client, modified_retailer_markets_client,
        modified_channels_regions_client, modified_channels_channels_client, modified_channels_markets_client,
        modified_regions_regions_client, modified_regions_channels_client, modified_regions_markets_client,
        modified_cust_regions_client, modified_cust_channels_client, modified_cust_markets_client
    ] 

    results = merge_regional_national(dfs, modified_national, modified_national_client)

    # Unpack all 18 results
    (
        modified_retailer_regions, modified_retailer_channels, modified_retailer_markets,
        modified_channels_regions, modified_channels_channels, modified_channels_markets,
        modified_regions_regions, modified_regions_channels, modified_regions_markets,
        modified_cust_regions, modified_cust_channels, modified_cust_markets,
        modified_retailer_regions_client, modified_retailer_channels_client, modified_retailer_markets_client,
        modified_channels_regions_client, modified_channels_channels_client, modified_channels_markets_client,
        modified_regions_regions_client, modified_regions_channels_client, modified_regions_markets_client,
        modified_cust_regions_client, modified_cust_channels_client, modified_cust_markets_client
    ) = results

In [28]:
markets_name = ['regions_RET' , 'channels_RET','market_RET'  ,'regions_CHAN' ,'channels_CHAN', 'market_CHAN','regions_REG' ,'channels_REG', 'market_REG' ,'regions_CUST' ,'channels_CUST', 'market_CUST']
markets_val = [regions_RET , channels_RET,market_RET  ,regions_CHAN ,channels_CHAN, market_CHAN,regions_REG,channels_REG,market_REG ,regions_CUST ,channels_CUST, market_CUST]
markets_dfs = [modified_retailer_regions , modified_retailer_channels,modified_retailer_markets  ,modified_channels_regions ,modified_channels_channels,  modified_channels_markets,modified_regions_regions,modified_regions_channels,modified_regions_markets,modified_cust_regions ,modified_cust_channels, modified_cust_markets]
client_dfs = [modified_retailer_regions_client , modified_retailer_channels_client,modified_retailer_markets_client  ,modified_channels_regions_client ,modified_channels_channels_client,  modified_channels_markets_client,modified_regions_regions_client , modified_regions_channels_client,modified_regions_markets_client,modified_cust_regions_client ,modified_cust_channels_client, modified_cust_markets_client]
market_total=[dya_retailer_reg_client,dya_retailer_channel_client,dya_retailer_mkt_client,dya_channel_reg_client,dya_channel_channel_client,dya_channel_mkt_client,dya_region_reg_client,dya_region_channel_client,dya_region_mkt_client,dya_cust_reg_client,dya_cust_channel_client,dya_cust_mkt_client]
slide_by = ['Region','Channel','Market']*4
slide_for = [*['Retailer']*3,*['Channel']*3,*['Region']*3,*[customareas]*3]
market_list_Avg={}
duplication_num_Avg = []
section_name_Avg = []


for idx,val in enumerate(markets_val):
    if len(val)>0:
        market_list_Avg[markets_name[idx]]=[val,markets_dfs[idx],client_dfs[idx],slide_by[idx],slide_for[idx]]
        duplication_num_Avg.append(len(client_dfs[idx].keys()))
        section_name_Avg.append('Value Vs AvgPrice By '+slide_for[idx]+' For '+slide_by[idx])
section_number_Avg = len(market_list_Avg.keys())

### Slide 12 : Category Evolution Analysis 

In [29]:
def cleanSectorSegment(dfDic):
    catDic, totalDic = {}, {}
    for key, df in dfDic.items():
        df = df.copy()
        catName=df.columns[0]
        df = df.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
        df = df.applymap(lambda x: 0 if pd.isna(x) else x)

        df[catName] = df[catName].astype(str)
        addative = (df[~df[catName].str.contains(' Total')].sort_values('WoB %', ascending=False).head(13))

        total = df[df[catName].str.contains(' Total')]
        total = total[total['Value Share'] > 0.005]
        
        total[catName] = total[catName].str.replace(' Total', '')

        # Check if the total DataFrame is empty and if so, add a "Grand" total row with zeros
        if total.empty:
            zero_row = pd.Series([0] * (len(df.columns) - 1), index=df.columns[1:])
            zero_row[catName] = "Grand"
            total = pd.concat([total, zero_row.to_frame().T], ignore_index=True)
        if not addative.empty:
            catDic[key.replace('  ', ' ')] = addative
        if not total.empty:
            totalDic[key.replace('  ', ' ')] = total

    return catDic, totalDic


In [30]:
import pandas as pd

def align_dics(dict1, dict2):
    col1 = list(dict1.values())[0].columns[0]
    values1 = dict1[list(dict1.keys())[0]][col1].tolist()

    new_dict2 = {}

    for key2, df2 in dict2.items():
        df2 = df2.copy()
        col2 = df2.columns[0]
        df2 = df2[df2[col2].isin(values1)]
        missing = set(values1) - set(df2[col2])
        for m in missing:
            new_row = {col2: m}
            for c in df2.columns[1:]:
                new_row[c] = 0
            df2 = pd.concat([df2, pd.DataFrame([new_row])], ignore_index=True)
        df2[col2] = pd.Categorical(df2[col2], categories=values1, ordered=True)
        df2 = df2.sort_values(col2).reset_index(drop=True)
        new_dict2[key2] = df2

    return dict1, new_dict2


In [31]:
globals()[f"cea_sectors_{slides_Period}_new"],cea_dya_sector_new,cea_sector_clients_new,cea_sector_clients_total_new ={},{},{},{}
if sectors:
    globals()[f"cea_sectors_{slides_Period}_new"],cea_dya_sectors_new=cleanSectorSegment(globals()[f"Sector_{slides_Period}_dfs"])
    cea_sectors_clients_new,cea_sectors_clients_total_new=cleanSectorSegment(Sector_client_dfs_category)
    globals()[f"cea_sectors_{slides_Period}_new"],cea_sectors_clients_new=align_dics(globals()[f"cea_sectors_{slides_Period}_new"],cea_sectors_clients_new)

globals()[f"cea_segment_{slides_Period}_new"],cea_dya_segment_new,cea_segment_clients_new,cea_segment_clients_total_new ={},{},{},{}
if segments:
    globals()[f"cea_segment_{slides_Period}_new"],cea_dya_segment_new=cleanSectorSegment(globals()[f"Segment_{slides_Period}_dfs"])
    cea_segment_clients_new,cea_segment_clients_total_new=cleanSectorSegment(Segment_client_dfs_category)
    globals()[f"cea_segment_{slides_Period}_new"],cea_segment_clients_new=align_dics( globals()[f"cea_segment_{slides_Period}_new"],cea_segment_clients_new)
    

In [32]:
globals()[f"cea_subsegment_{slides_Period}_new"],cea_dya_subsegment_new,cea_subsegment_clients_new,cea_subsegment_clients_total_new ={},{},{},{}
if subsegments:
    globals()[f"cea_subsegment_{slides_Period}_new"],cea_dya_subsegment_new=cleanSectorSegment(globals()[f"SubSegment_{slides_Period}_dfs"])
    cea_subsegment_clients_new,cea_subsegment_clients_total_new=cleanSectorSegment(SubSegment_client_dfs_category)
    globals()[f"cea_subsegment_{slides_Period}_new"],cea_subsegment_clients_new=align_dics(globals()[f"cea_subsegment_{slides_Period}_new"],cea_subsegment_clients_new)
    

globals()[f"cea_subcategory_{slides_Period}_new"],cea_dya_subcategory_new,cea_subcategory_clients_new,cea_subcategory_clients_total_new ={},{},{},{}
if subcategories:
    globals()[f"cea_subcategory_{slides_Period}_new"],cea_dya_subcategory_new=cleanSectorSegment(globals()[f"SubCategory_{slides_Period}_dfs"])
    cea_subcategory_clients_new,cea_subcategory_clients_total_new=cleanSectorSegment(SubCategory_client_dfs_category)
    globals()[f"cea_subcategory_{slides_Period}_new"],cea_subcategory_clients_new=align_dics( globals()[f"cea_subcategory_{slides_Period}_new"],cea_subcategory_clients_new)
    

## Slide 13 : Share And Growth

In [33]:
retailerRegion = {}
retailerChannel = {}
retailertMarket = {}

channelRegion = {}
channelChannel = {}
channelMarket = {}

regionRegion = {}
regionChannel = {}
regionMarket = {}

custRegion = {}
custChannel = {}
custMarket = {}

retailer_reg_Total={}
retailer_chan_Total={}
retailer_mkt_Total={}

channel_reg_Total={}
channel_chan_Total={}
channel_mkt_Total={}

region_reg_Total={}
region_chan_Total={}
region_mkt_Total={}

cust_reg_Total={}
cust_chan_Total={}
cust_mkt_Total={}
#********** Retailer *********
if len(retailers) != 0:
    retailerRegion,retailer_reg_Total = grandtot(globals().get("modified_retailer_Region", {}) or {},regions_RET)
    retailerChannel ,retailer_chan_Total= grandtot(globals().get("modified_retailer_Channel", {}) or {},channels_RET)
    retailertMarket,retailer_mkt_Total =grandtot(globals().get("modified_retailer_Market", {}) or {},market_RET)
# ********* Channels *********
if len(channels) != 0:
    channelRegion,channel_reg_Total = grandtot(globals().get("modified_channels_Region", {}) or {},regions_CHAN)
    channelChannel,channel_chan_Total = grandtot(globals().get("modified_channels_Channel", {}) or {},channels_CHAN)
    channelMarket,channel_mkt_Total = grandtot(globals().get("modified_channels_Market", {}) or {},market_CHAN)
# ********* Regions **********
if len(regions) != 0:
    regionRegion,region_reg_Total = grandtot(globals().get("modified_regions_Region", {}) or {},regions_REG)
    regionChannel,region_chan_Total = grandtot(globals().get("modified_regions_Channel", {}) or {},channels_REG)
    regionMarket,region_mkt_Total = grandtot(globals().get("modified_regions_Market", {}) or {},market_REG)
# ******** Custom ********
if len(cust) != 0: 
    custRegion,cust_reg_Total = grandtot(globals().get("modified_cust_Region", {}) or {},regions_CUST)
    custChannel,cust_chan_Total =grandtot(globals().get("modified_cust_Channel", {}) or {},channels_CUST)
    custMarket,cust_mkt_Total = grandtot(globals().get("modified_cust_Market", {}) or {},market_CUST)


In [34]:
# Ensure final dictionaries are initialized
clientRetailerRegion = {}
clientretailerChannel = {}
clientRetailertMarket = {}

clientChannelRegion = {}
clientChannelChannel = {}
clientChannelMarket = {}

clientRegionRegion = {}
clientRegionChannel = {}
clientRegionMarket = {}

clientCustRegion = {}
clientCustChannel = {}
clientCustMarket = {}


retailer_reg_ClientTotal={}
retailer_chan_ClientTotal={}
retailer_mkt_ClientTotal={}

channel_reg_ClientTotal={}
channel_chan_ClientTotal={}
channel_mkt_ClientTotal={}

region_reg_ClientTotal={}
region_chan_ClientTotal={}
region_mkt_ClientTotal={}

cust_reg_ClientTotal={}
cust_chan_ClientTotal={}
cust_mkt_ClientTotal={}
#********** Retailer *********
if len(retailers) != 0:
    clientRetailerRegion,retailer_reg_ClientTotal= grandtot(globals().get("modified_retailer_Region_client", {}) or {},regions_RET)
    clientretailerChannel,retailer_chan_ClientTotal =grandtot(globals().get("modified_retailer_Channel_client", {}) or {},channels_RET)
    clientRetailertMarket,retailer_mkt_ClientTotal = grandtot(globals().get("modified_retailer_Market_client", {}) or {},market_RET)
# ********* Channels **********
if len(channels) != 0:
    clientChannelRegion,channel_reg_ClientTotal = grandtot(globals().get("modified_channels_Region_client", {}) or {},regions_CHAN)
    clientChannelChannel,channel_chan_ClientTotal= grandtot(globals().get("modified_channels_Channel_client", {}) or {},channels_CHAN)
    clientChannelMarket,channel_mkt_ClientTotal= grandtot(globals().get("modified_channels_Market_client", {}) or {},market_CHAN)
# ********* Regions **********
if len(regions) != 0:
    clientRegionRegion,region_reg_ClientTotal = grandtot(globals().get("modified_regions_Region_client", {}) or {},regions_REG)
    clientRegionChannel,region_chan_ClientTotal = grandtot(globals().get("modified_regions_Channel_client", {}) or {},channels_REG)
    clientRegionMarket,region_mkt_ClientTotal = grandtot(globals().get("modified_regions_Market_client", {}) or {},market_REG)
# ******** POS ********
if len(cust) != 0: 
    clientCustRegion,cust_reg_ClientTotal =grandtot(globals().get("modified_cust_Region_client", {}) or {},regions_CUST)
    clientCustChannel,cust_chan_ClientTotal = grandtot(globals().get("modified_cust_Channel_client", {}) or {},channels_CUST)
    clientCustMarket,cust_mkt_ClientTotal = grandtot(globals().get("modified_cust_Market_client", {}) or {},market_CUST)
 


In [35]:
retailerCatDic,retailerSecDic,retailerSegDic,retailerSubSegDic,retailerSubCatDic=[],[],[],[],[]
channelCatDic,channelSecDic,channelSegDic,channelSubSegDic,channelSubCatDic=[],[],[],[],[]
regionCatDic,regionSecDic,regionSegDic,regionSubSegDic,regionSubCatDic=[],[],[],[],[]
custCatDic,custSecDic,custSegDic,custSubSegDic,custSubCatDic=[],[],[],[],[]

# if modified_retailer_Region:

retailerCatDic = [key for key in retailer_reg_ClientTotal.keys() if (len(set(categories).intersection(set(key.split(' | '))))) ]
retailerSecDic = [key for key in retailer_reg_ClientTotal.keys() if len(set(sectors).intersection(set(key.split(' | '))))]
retailerSegDic = [key for key in retailer_reg_ClientTotal.keys() if len(set(segments).intersection(set(key.split(' | '))))]
retailerSubSegDic = [key for key in retailer_reg_ClientTotal.keys() if len(set(subsegments).intersection(set(key.split(' | '))))]
retailerSubCatDic = [key for key in retailer_reg_ClientTotal.keys() if len(set(subcategories).intersection(set(key.split(' | '))))]



# if modified_channels_Region:

channelCatDic = [key for key in channel_reg_ClientTotal.keys() if len(set(categories).intersection(set(key.split(' | '))))]
channelSecDic = [key for key in channel_reg_ClientTotal.keys() if len(set(sectors).intersection(set(key.split(' | '))))]
channelSegDic = [key for key in channel_reg_ClientTotal.keys() if len(set(segments).intersection(set(key.split(' | '))))]
channelSubSegDic = [key for key in channel_reg_ClientTotal.keys() if len(set(subsegments).intersection(set(key.split(' | '))))]
channelSubCatDic = [key for key in channel_reg_ClientTotal.keys() if len(set(subcategories).intersection(set(key.split(' | '))))]

# if modified_regions_Region:

regionCatDic = [key for key in region_reg_ClientTotal.keys() if len(set(categories).intersection(set(key.split(' | '))))]
regionSecDic = [key for key in region_reg_ClientTotal.keys() if len(set(sectors).intersection(set(key.split(' | '))))]
regionSegDic = [key for key in region_reg_ClientTotal.keys() if len(set(segments).intersection(set(key.split(' | '))))]
regionSubSegDic = [key for key in region_reg_ClientTotal.keys() if len(set(subsegments).intersection(set(key.split(' | '))))]
regionSubCatDic = [key for key in region_reg_ClientTotal.keys() if len(set(subcategories).intersection(set(key.split(' | '))))]


# if cust_dfs_new:

custCatDic = [key for key in cust_reg_ClientTotal.keys() if len(set(categories).intersection(set(key.split(' | ')))) ]
custSecDic = [key for key in cust_reg_ClientTotal.keys() if len(set(sectors).intersection(set(key.split(' | ')))) ]
custSegDic = [key for key in cust_reg_ClientTotal.keys() if len(set(segments).intersection(set(key.split(' | '))))]
custSubSegDic = [key for key in cust_reg_ClientTotal.keys() if len(set(subsegments).intersection(set(key.split(' | '))))]
custSubCatDic = [key for key in cust_reg_ClientTotal.keys() if len(set(subcategories).intersection(set(key.split(' | '))))]



In [36]:
retailerField = {'Region':[retailerRegion,clientRetailerRegion,retailer_reg_Total,retailer_reg_ClientTotal],'Channel':[retailerChannel,clientretailerChannel,retailer_chan_Total,retailer_chan_ClientTotal],'Market':[retailertMarket,clientRetailertMarket,retailer_mkt_Total,retailer_mkt_ClientTotal]}
channelField = {'Region':[channelRegion,clientChannelRegion,channel_reg_Total,channel_reg_ClientTotal],'Channel':[channelChannel,clientChannelChannel,channel_chan_Total,channel_chan_ClientTotal],'Market':[channelMarket,clientChannelMarket,channel_mkt_Total,channel_mkt_ClientTotal]}
regionField = {'Region':[regionRegion,clientRegionRegion,region_reg_Total,region_reg_ClientTotal],'Channel':[regionChannel,clientRegionChannel,region_chan_Total,region_chan_ClientTotal],'Market':[regionMarket,clientRegionMarket,region_mkt_Total,region_mkt_ClientTotal]}
custField = {'Region':[custRegion,clientCustRegion,cust_reg_Total,cust_reg_ClientTotal],'Channel':[custChannel,clientCustChannel,cust_chan_Total,cust_chan_ClientTotal],'Market':[custMarket,clientCustMarket,cust_mkt_Total,cust_mkt_ClientTotal]}

### Slide 14: Momentum Analysis

In [37]:
def merge_dicts(target, *sources):
    for source in sources:
        if source:  # Ensure the source is not None or empty
            target.update(source)

modified_momentum_retailers_new = {}
merge_dicts(
    modified_momentum_retailers_new, 
    globals().get('modified_momentum_retailer_Region', {}), 
    globals().get('modified_momentum_retailer_Channel', {}), 
    globals().get('modified_momentum_retailer_Market', {})
)

modified_momentum_channels_new = {}
merge_dicts(
    modified_momentum_channels_new, 
    globals().get('modified_momentum_channels_Region', {}), 
    globals().get('modified_momentum_channels_Channel', {}), 
    globals().get('modified_momentum_channels_Market', {})
)

modified_momentum_regions_new = {}
merge_dicts(
    modified_momentum_regions_new, 
    globals().get('modified_momentum_regions_Region', {}), 
    globals().get('modified_momentum_regions_Channel', {}), 
    globals().get('modified_momentum_regions_Market', {})
)

modified_momentum_cust_new = {}
merge_dicts(
    modified_momentum_cust_new, 
    globals().get('modified_momentum_cust_Region', {}), 
    globals().get('modified_momentum_cust_Channel', {}), 
    globals().get('modified_momentum_cust_Market', {})
)


In [38]:
def process_dateMomentum(manuf_dfs, datacolumn, numskpcol,hirlis=[],market=False):
    output = {}  
    for k, df in manuf_dfs.items():
        df = df.copy()
        if not df.empty:
            # Ensure numskpcol is valid
            if numskpcol not in [2, 4]:
                raise ValueError("numskpcol must be either 2 or 4")
            
            # Extract the first numskpcol columns as index
            index_cols = list(df.columns[:numskpcol])
            
            # Pivot the DataFrame
            df_pivot = df.pivot_table(index=index_cols, 
                                    columns=datacolumn, 
                                    aggfunc='sum', 
                                    dropna=False)
            df_pivot.columns = [f"{metric} {year}" for metric, year in df_pivot.columns]
            df_pivot.reset_index(inplace=True)
            df_pivot = df_pivot.dropna(subset=df_pivot.columns[numskpcol:], how='all')
            numeric_cols = df_pivot.select_dtypes(include=['number']).columns
            years = sorted(df[datacolumn].dropna().unique(), key=list(df[datacolumn].dropna()).index)
            metrics = sorted(
                set(col.rsplit(" ", 1)[0] for col in numeric_cols),
                key=lambda x: list(df.columns).index(x) if x in df.columns else float('inf')
            )
            sorted_columns = index_cols + [f"{metric} {year}" for year in years for metric in metrics]
            df_pivot = df_pivot[[col for col in sorted_columns if col in df_pivot.columns]]
            if market==True:
                df_pivot = df_pivot[df_pivot[df_pivot.columns[1]].isin(hirlis)]

            for metric in metrics:
                matching_cols = [col for col in numeric_cols if col.startswith(metric)]
                if matching_cols:
                    if "Sales" in metric:
                        df_pivot[f"Total {metric}"] = df_pivot[matching_cols].sum(axis=1)
                    else:
                        df_pivot[f"Total {metric}"] = df_pivot[matching_cols].mean(axis=1)
            
            df_pivot = df_pivot.loc[:, ~df_pivot.columns.astype(str).str.contains("nan", case=False, na=False)]
            comp = df_pivot[~df_pivot[f"{BrandOrTopB}"].isin(client_brands)]
            value_sales_cols = [col for col in comp.columns if "Value Sales P12M" in col]

            # Get the last (latest) one
            latest_vs_col = value_sales_cols

            # Proceed only if such a column exists
            if latest_vs_col:
                top5_competitors = comp.sort_values(by=latest_vs_col, ascending=False).head(6)
                top5_list = top5_competitors[f"{BrandOrTopB}"].tolist()

            else:
                print("No Value Sales column found for sorting.")
            t_filtered = df_pivot[df_pivot[f"{BrandOrTopB}"].isin(client_brands) |
                        ((df_pivot[f"{BrandOrTopB}"].isin(top5_list)) & (df_pivot['Value Share P12M'] > 0.02))]
            t_filtered['Value Sales P12M'] = t_filtered['Value Sales P12M'].astype(float)
            t_filtered = t_filtered.sort_values('Value Sales P12M', ascending=False)

            
                                
            output[k] = t_filtered

    return output


In [39]:
modified_momentum_retailersfinal=process_dateMomentum(modified_momentum_retailers_new,"Time Period",2,retailers,market=True)
modified_momentum_channelsfinal=process_dateMomentum(modified_momentum_channels_new,"Time Period",2,channels,market=True)
modified_momentum_regionsfinal=process_dateMomentum(modified_momentum_regions_new,"Time Period",2,regions,market=True)
modified_momentum_custfinal=process_dateMomentum(modified_momentum_cust_new,"Time Period",2,cust,market=True)

In [40]:
def add_totals_after_grouping(data_dict):
    updated_dict = {}

    for key, df in data_dict.items():
        if not df.empty:
            first_col = df.columns[0]  # Get the first column name
            df=df[df[first_col]!="Grand Total"]
            numeric_cols = df.select_dtypes(include=['number']).columns  # Get numeric columns

            grouped = df.groupby(first_col)[numeric_cols].sum().reset_index()  # Sum only numeric columns

            new_df_list = []
            for group, group_df in df.groupby(first_col):
                new_df_list.append(group_df)  # Append original group
                
                total_row = grouped[grouped[first_col] == group].copy()  # Get total row
                total_row[first_col] = f"{group} Total"  # Rename the group name for total row
                new_df_list.append(total_row)  # Append total row

            updated_dict[key] = pd.concat(new_df_list, ignore_index=True)  # Combine results

    return updated_dict
variables = [
    "modified_momentum_retailersfinal",
    "modified_momentum_channelsfinal",
    "modified_momentum_regionsfinal",
    "modified_momentum_custfinal"
]

for var in variables:
    if var in globals():
        globals()[var] = add_totals_after_grouping(globals()[var])


In [41]:
markets_val = [regions_RET + channels_RET+ market_RET  +regions_CHAN +channels_CHAN+ market_CHAN +regions_REG+channels_REG+market_REG+regions_CUST +channels_CUST+ market_CUST]
markets_val = markets_val[0]
def process_RET_CHAN_Momentum(dataframes, client_brands,areaby,col=" ",lst=[],sec=False):
    # Dictionary to hold all modified DataFrames in a single structure
    modified_data = {}
    for k, df in dataframes.items():
        # Copy and process each DataFrame
        t = df.copy()
        if not t.empty:
            t = t[~t[f"{BrandOrTopB}"].fillna("").str.strip().eq("")]
            top_brands_totals = t[(t[f"{BrandOrTopB}"].str.contains("Total")) & (t[f"{BrandOrTopB}"]!="Grand Total") & (~t[f"{BrandOrTopB}"].isin([i+" Total" for i in client_brands]))]
            competitors = top_brands_totals.sort_values(by = "Value Sales P12M",ascending=False).head(5)[f"{BrandOrTopB}"].to_list()
            competitors = [i.replace(" Total","") for i in competitors]
            t_filtered = t[t[f"{BrandOrTopB}"].isin(client_brands) |
                                    ((t[f"{BrandOrTopB}"].isin(competitors)) & (t['Value Share P12M'] > 0.02))]
            t_filtered = t_filtered.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
            t_filtered['Value Sales P12M'] = t_filtered['Value Sales P12M'].astype(float)
            t_filtered = t_filtered.sort_values('Value Sales P12M', ascending=False)
            t_filtered = t_filtered[t_filtered.iloc[:, 1].isin(markets_val)]
            if sec and col and col in t_filtered.columns:
                t_filtered = t_filtered[t_filtered[col].isin(lst)]
            filtered_df = t_filtered
            # newKey=k.split(' | ')[0] +" | " +f'{areaby}'
            if not filtered_df.empty:
                if sec==True:
                    filtered_df['Labels'] = filtered_df[f"{BrandOrTopB}"] + "/" + filtered_df[filtered_df.columns[1]]+"/" + filtered_df["Sector"]+ "/" + filtered_df[f"{col}"]  
                else :    
                    filtered_df['Labels'] = filtered_df[f"{BrandOrTopB}"] + "/" + filtered_df[filtered_df.columns[1]]

                modified_data[k] = pd.concat([modified_data.get(k, pd.DataFrame()), filtered_df], ignore_index=True)
                modified_data[k] = modified_data[k].sort_values('Value Sales P12M', ascending=False)
                modified_data[k] = modified_data[k].drop_duplicates() 
    return modified_data

In [42]:
modified_momentum_retailers = process_RET_CHAN_Momentum(modified_momentum_retailersfinal,client_brands,"Retailers")

modified_momentum_channels = process_RET_CHAN_Momentum(modified_momentum_channelsfinal,client_brands,"Channels")

modified_momentum_regions = process_RET_CHAN_Momentum(modified_momentum_regionsfinal,client_brands,"Regions")

modified_momentum_cust = process_RET_CHAN_Momentum(modified_momentum_custfinal,client_brands, f"{customareas}")


In [43]:
def merge_dicts(target, *sources):
    for source in sources:
        if source:  # Ensure the source is not None or empty
            target.update(source)
if segments:
    modified_momentum_retailers_seg_new = {}
    merge_dicts(
        modified_momentum_retailers_seg_new, 
        globals().get('modified_momentum_seg_retailer_Region', {}), 
        globals().get('modified_momentum_seg_retailer_Channel', {}), 
        globals().get('modified_momentum_seg_retailer_Market', {})
    )

    modified_momentum_channels_seg_new = {}
    merge_dicts(
        modified_momentum_channels_seg_new, 
        globals().get('modified_momentum_seg_channels_Region', {}), 
        globals().get('modified_momentum_seg_channels_Channel', {}), 
        globals().get('modified_momentum_seg_channels_Market', {})
    )

    modified_momentum_regions_seg_new = {}
    merge_dicts(
        modified_momentum_regions_seg_new, 
        globals().get('modified_momentum_seg_regions_Region', {}), 
        globals().get('modified_momentum_seg_regions_Channel', {}), 
        globals().get('modified_momentum_seg_regions_Market', {})
    )

    modified_momentum_cust_seg_new = {}
    merge_dicts(
        modified_momentum_cust_seg_new, 
        globals().get('modified_momentum_seg_cust_Region', {}), 
        globals().get('modified_momentum_seg_cust_Channel', {}), 
        globals().get('modified_momentum_seg_cust_Market', {})
    )
    
###### SubSegment    
if subsegments:
    modified_momentum_retailers_subseg_new = {}
    merge_dicts(
        modified_momentum_retailers_subseg_new, 
        globals().get('modified_momentum_subseg_retailer_Region', {}), 
        globals().get('modified_momentum_subseg_retailer_Channel', {}), 
        globals().get('modified_momentum_subseg_retailer_Market', {})
    )

    modified_momentum_channels_subseg_new = {}
    merge_dicts(
        modified_momentum_channels_subseg_new, 
        globals().get('modified_momentum_subseg_channels_Region', {}), 
        globals().get('modified_momentum_subseg_channels_Channel', {}), 
        globals().get('modified_momentum_subseg_channels_Market', {})
    )

    modified_momentum_regions_subseg_new = {}
    merge_dicts(
        modified_momentum_regions_subseg_new, 
        globals().get('modified_momentum_subseg_regions_Region', {}), 
        globals().get('modified_momentum_subseg_regions_Channel', {}), 
        globals().get('modified_momentum_subseg_regions_Market', {})
    ) 

    modified_momentum_cust_subseg_new = {}
    merge_dicts(
        modified_momentum_cust_subseg_new, 
        globals().get('modified_momentum_subseg_cust_Region', {}), 
        globals().get('modified_momentum_subseg_cust_Channel', {}), 
        globals().get('modified_momentum_subseg_cust_Market', {})
    )    
###### SubCategory   
if subcategories:
    modified_momentum_retailers_subcatg_new= {}
    merge_dicts(
        modified_momentum_retailers_subcatg_new, 
        globals().get('modified_momentum_subcat_retailer_Region', {}), 
        globals().get('modified_momentum_subcat_retailer_Channel', {}), 
        globals().get('modified_momentum_subcat_retailer_Market', {})
    )

    modified_momentum_channels_subcatg_new = {}
    merge_dicts(
        modified_momentum_channels_subcatg_new, 
        globals().get('modified_momentum_subcat_channels_Region', {}), 
        globals().get('modified_momentum_subcat_channels_Channel', {}), 
        globals().get('modified_momentum_subcat_channels_Market', {})
    )

    modified_momentum_regions_subcatg_new = {}
    merge_dicts(
        modified_momentum_regions_subcatg_new, 
        globals().get('modified_momentum_subcat_regions_Region', {}), 
        globals().get('modified_momentum_subcat_regions_Channel', {}), 
        globals().get('modified_momentum_subcat_regions_Market', {})
    )

    modified_momentum_cust_subcatg_new = {}
    merge_dicts(
        modified_momentum_cust_subcatg_new, 
        globals().get('modified_momentum_subcat_cust_Region', {}), 
        globals().get('modified_momentum_subcat_cust_Channel', {}), 
        globals().get('modified_momentum_subcat_cust_Market', {})
    )       

In [44]:
if segments:   
    modified_momentum_seg_retailersfinal=process_dateMomentum(modified_momentum_retailers_seg_new,"Time Period",4,retailers,market=True)
    modified_momentum_seg_channelsfinal=process_dateMomentum(modified_momentum_channels_seg_new,"Time Period",4,channels,market=True)
    modified_momentum_seg_regionsfinal=process_dateMomentum(modified_momentum_regions_seg_new,"Time Period",4,regions,market=True)
    modified_momentum_seg_custfinal=process_dateMomentum(modified_momentum_cust_seg_new,"Time Period",4,cust,market=True)
if subsegments:   
    modified_momentum_subseg_retailersfinal=process_dateMomentum(modified_momentum_retailers_subseg_new,"Time Period",4,retailers,market=True)
    modified_momentum_subseg_channelsfinal=process_dateMomentum(modified_momentum_channels_subseg_new,"Time Period",4,channels,market=True)
    modified_momentum_subseg_regionsfinal=process_dateMomentum(modified_momentum_region_subseg_new,"Time Period",4,regions,market=True)
    modified_momentum_subseg_custfinal=process_dateMomentum(modified_momentum_cust_subseg_new,"Time Period",4,cust,market=True)
if subcategories:   
    modified_momentum_subcat_retailersfinal=process_dateMomentum(modified_momentum_retailers_subcatg_new,"Time Period",4,retailers,market=True)
    modified_momentum_subcat_channelsfinal=process_dateMomentum(modified_momentum_channels_subcatg_new,"Time Period",4,channels,market=True)
    modified_momentum_subcat_regionsfinal=process_dateMomentum(modified_momentum_regions_subcatg_new,"Time Period",4,regions,market=True)
    modified_momentum_subcat_custfinal=process_dateMomentum(modified_momentum_cust_subcatg_new,"Time Period",4,cust,market=True)        

In [45]:
##### Grouping & Total
variables = []

if segments:
    variables.extend([
        "modified_momentum_seg_retailersfinal",
        "modified_momentum_seg_channelsfinal",
        "modified_momentum_seg_regionsfinal",
        "modified_momentum_seg_custfinal"
    ])

if subsegments:
    variables.extend([
        "modified_momentum_subseg_retailersfinal",
        "modified_momentum_subseg_channelsfinal",
        "modified_momentum_subseg_regionsfinal",
        "modified_momentum_subseg_custfinal"
    ])

if subcategories:
    variables.extend([
        "modified_momentum_subcat_retailersfinal",
        "modified_momentum_subcat_channelsfinal",
        "modified_momentum_subcat_regionsfinal",
        "modified_momentum_subcat_custfinal"
    ])

for var in variables:
    if var in globals() and globals()[var] is not None:
        globals()[var] = add_totals_after_grouping(globals()[var])


In [46]:
if segments:
    modified_momentum_retailers_seg = process_RET_CHAN_Momentum(modified_momentum_seg_retailersfinal,client_brands,"Retailers","Segment", segments,sec=True)
    modified_momentum_channels_seg = process_RET_CHAN_Momentum(modified_momentum_seg_channelsfinal,client_brands,"Channels","Segment", segments,sec=True)
    modified_momentum_regions_seg = process_RET_CHAN_Momentum(modified_momentum_seg_regionsfinal,client_brands,"Regions","Segment", segments,sec=True)
    modified_momentum_cust_seg = process_RET_CHAN_Momentum(modified_momentum_seg_custfinal,client_brands,f"{customareas}","Segment", segments,sec=True)
    
if subsegments:
    modified_momentum_retailers_subseg = process_RET_CHAN_Momentum(modified_momentum_subseg_retailersfinal,client_brands,"Retailers","SubSegment", subsegments,sec=True)
    modified_momentum_channels_subseg = process_RET_CHAN_Momentum(modified_momentum_subseg_channelsfinal,client_brands,"Channels","SubSegment", subsegments,sec=True)
    modified_momentum_regions_subseg = process_RET_CHAN_Momentum(modified_momentum_subseg_regionsfinal,client_brands,"Regions","SubSegment", subsegments,sec=True)
    modified_momentum_cust_subseg = process_RET_CHAN_Momentum(modified_momentum_subseg_custfinal,client_brands,f"{customareas}","SubSegment", subsegments,sec=True)
    
if subcategories:
    modified_momentum_retailers_subcatg = process_RET_CHAN_Momentum(modified_momentum_subcat_retailersfinal,client_brands,"Retailers","SubCategory", subcategories,sec=True)
    modified_momentum_channels_subcatg = process_RET_CHAN_Momentum(modified_momentum_subcat_channelsfinal,client_brands,"Channels","SubCategory", subcategories,sec=True)
    modified_momentum_regions_subcatg = process_RET_CHAN_Momentum(modified_momentum_subcat_regionsfinal,client_brands,"Regions","SubCategory", subcategories,sec=True)
    modified_momentum_cust_subcatg = process_RET_CHAN_Momentum(modified_momentum_subcat_custfinal,client_brands,f"{customareas}","SubCategory", subcategories,sec=True)
        

In [47]:
momentum_dfs_frist=process_dateMomentum(momentum_dfs,"Time Period",2,market=False)
momentum_dfs_new=add_totals_after_grouping(momentum_dfs_frist)


In [48]:
modified_momentum_dfs = {}
for key in momentum_dfs_new.keys():
    t = momentum_dfs_new[key].copy()
    if not t.empty:
        t[f'{BrandOrTopB}'] = t[f'{BrandOrTopB}'].ffill()
        if not t.empty:
            top_brands_totals = t[(t[f'{BrandOrTopB}'].str.contains("Total")) & (t[f'{BrandOrTopB}']!="Grand Total|All Others Total") & (~t[f"{BrandOrTopB}"].isin([i+" Total" for i in client_brands]))]
            competitors = top_brands_totals.sort_values(by = "Value Sales P12M",ascending=False).head(6)[f'{BrandOrTopB}'].to_list()
            competitors = [i.replace(" Total","") for i in competitors]
            t_filtered = t[t[f'{BrandOrTopB}'].isin(client_brands) |
                                    ((t[f'{BrandOrTopB}'].isin(competitors)) & (t['Value Share P12M'] > 0.02))]

            t_filtered = t_filtered.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
            t_filtered['Value Sales P12M'] = t_filtered['Value Sales P12M'].astype(float)
            t_filtered = t_filtered.sort_values('Value Sales P12M', ascending=False)
            t_filtered['Labels'] = t_filtered[f'{BrandOrTopB}'] + "/" + t_filtered['Sector'] 

            if not t_filtered.empty:
                modified_momentum_dfs[key] = t_filtered 

In [49]:
sharGrowthDf_sec

{'NATIONAL': ['Soft Cheese', 'Aperitif', 'Ingredient A Chaud'],
 'Carrefour': ['Soft Cheese', 'Aperitif', 'Ingredient A Chaud'],
 'Intermarche': ['Soft Cheese', 'Aperitif', 'Ingredient A Chaud']}

In [50]:
def genrateIndexList(scopeCategory ,chartIndex =0 ,chartCount =4 ):
    if len(scopeCategory) == 0:
        return []
    page = len(scopeCategory) # number of sector or segment or category
 
    lis=[]
    while page >chartCount:
        lis.append(chartCount + chartIndex)
        page -=chartCount
    lis.append(page + chartIndex)
    return [lis]

sectorIndex = []
segmentIndex = []
subsegmentIndex = []
subcategoryIndex = []
retailerREGIndex = []
retailerCHAIndex = []
retailerMARIndex = []
channelREGIndex = []
channelCHAIndex = []
channelMARIndex = []
regionREGIndex = []
regionCHAIndex = []
regionMARIndex = []
custREGIndex = []
custCHAIndex = []
custMARIndex = []

if len (sectors)!=0:
    
    for key,value in sharGrowthDf_sec.items():
        sectorIndex+=genrateIndexList(value,4,3)[0]

if len(segments)!=0:
    
    for key,value in sharGrowthDf_seg.items():
        segmentIndex+=genrateIndexList(value,4,3)[0]

if len(subsegments)!=0:
    
    for key,value in sharGrowthDf_subseg.items():
        subsegmentIndex+=genrateIndexList(value,4,3)[0]

if len(subcategories)!=0:
    
    for key,value in sharGrowthDf_subcat.items():
        subcategoryIndex+=genrateIndexList(value,4,3)[0]

if len(regions_RET)!=0:
    for key,value in shareGrowthDf_retailers_REG.items():
        retailerREGIndex+=genrateIndexList(value,4,3)[0]

if len(channels_RET)!=0:
    for key,value in shareGrowthDf_retailers_CHAN.items():
        retailerCHAIndex+=genrateIndexList(value,4,3)[0]
if len(market_RET)!=0:
    for key,value in shareGrowthDf_retailers_MAR.items():
        retailerMARIndex+=genrateIndexList(value,4,3)[0]
        
if len(regions_CHAN)!=0:
    for key,value in shareGrowthDf_channels_REG.items():
        channelREGIndex+=genrateIndexList(value,4,3)[0]
if len(channels_CHAN)!=0:
    for key,value in shareGrowthDf_channels_CHAN.items():
        channelCHAIndex+=genrateIndexList(value,4,3)[0]
if len(market_CHAN)!=0:
    for key,value in shareGrowthDf_channels_MAR.items():
        channelMARIndex+=genrateIndexList(value,4,3)[0]

if len(regions_REG)!=0:
    for key,value in shareGrowthDf_regions_REG.items():
        regionREGIndex+=genrateIndexList(value,4,3)[0]
if len(channels_REG)!=0:
    for key,value in shareGrowthDf_regions_CHAN.items():
        regionCHAIndex+=genrateIndexList(value,4,3)[0]
if len(market_REG)!=0:
    for key,value in shareGrowthDf_regions_MAR.items():
        regionMARIndex+=genrateIndexList(value,4,3)[0]


if len(regions_CUST)!=0:
    for key,value in shareGrowthDf_custom_REG.items():
        custREGIndex+=genrateIndexList(value,4,3)[0]
if len(channels_CUST)!=0:
    for key,value in shareGrowthDf_custom_CHAN.items():
        custCHAIndex+=genrateIndexList(value,4,3)[0]
if len(market_CUST)!=0:
    for key,value in shareGrowthDf_custom_MAR.items():
        custMARIndex+=genrateIndexList(value,4,3)[0]


# Slide Creations

## First Landscape Deck

In [51]:
index = [
    *[0]*(2+(1 if sectors else 0)+(1 if segments else 0)+(1 if subcategories else 0)+(1 if subsegments else 0)),
           *[1]*(2+(1 if sectors else 0)+(1 if segments else 0)+(1 if subcategories else 0)+(1 if subsegments else 0)),
            #   *[2]*((1 if sectors else 0) +(1 if segments else 0)+(1 if subcategories else 0)+(1 if subsegments else 0)),
            #   *[2]*section_number,
               *[3]*((1 if sectors else 0) +(1 if segments else 0)+(1 if subcategories else 0)+(1 if subsegments else 0)),
             *[3]*section_number_Avg,
                ]
index=[i for i in index if i !=[]]

duplication_1 = [len(modified_manuf_dfs_new.keys()), len(modified_brands_share_new.keys()),len(modified_sectors_dfs_new.keys())if sectors else 0,len(modified_segment_dfs_new.keys())if segments else 0,len(modified_subsegment_dfs_new.keys())if subsegments else 0,len(modified_subcategories_dfs_new.keys())if subcategories else 0]
duplication_2 = [len(modified_manuf_dfs_new.keys()), len(modified_brands_share_new.keys()), len(modified_sectors_dfs_new.keys())if sectors else 0, len(modified_segment_dfs_new.keys())if segments else 0,len(modified_subsegment_dfs_new.keys())if subsegments else 0,len(modified_subcategories_dfs_new.keys())if subcategories else 0]
#duplication_3 = [len(modified_sectors_P12M_new.keys()) if sectors else 0, len(modified_segment_P12M_new.keys())if segments else 0, len(modified_subsegment_P12M_new.keys())if subsegments else 0, len(modified_subcategories_P12M_new.keys())if subcategories else 0,*duplication_num]
duplication_4 = [len(modified_sectors_clients_new.keys())if sectors else 0,len(modified_segment_clients_new.keys())if segments else 0, len(modified_subsegment_clients_new.keys())if subsegments else 0, len(modified_subcategories_clients_new.keys())if subcategories else 0, *duplication_num_Avg]

duplication =  duplication_1 + duplication_2 + duplication_4 #+ duplication_3 # 
duplication = [item for item in duplication if item !=0]

section_names_slide1 = ["Market Trends by Manufacturer","Market Trends by Brands","Market Trends by Sectors"] + (["Market Trends by Segments"] if len(segments)>0 else [])+(["Market Trends by SubSegments"] if len(subsegments)>0 else [])+(["Market Trends by SubCategory"] if len(subcategories)>0 else [])
section_names_slide2 = ["Market Concentration By Manufacturer", "Market Concentration By Brands", "Market Concentration By Sectors"]+ (["Market Concentration By Segments"] if len(segments)>0 else [])+(["Market Concentration by SubSegments"] if len(subsegments)>0 else [])+(["Market Concentration by SubCategory"] if len(subcategories)>0 else [])
#section_names_slide3 = (["Market Growth By Sectors"]if len(sectors)>0 else [])+(["Market Growth By Segments"]if len(segments)>0 else [])+(["Market Growth By SubSegments"]if len(subsegments)>0 else [])+(["Market Growth By SubCategory"]if len(subcategories)>0 else [])+[*section_name_Growth]
section_names_slide4 = (["Value Vs AvgPrice By Sectors"]if len(sectors)>0 else [])+(["Value Vs AvgPrice By Segments"]if len(segments)>0 else [])+(["Value Vs AvgPrice By SubSegments"]if len(subsegments)>0 else [])+(["Value Vs AvgPrice By SubCategory"]if len(subcategories)>0 else [])+[*section_name_Avg]


section_names = [*section_names_slide1 ,*section_names_slide2,*section_names_slide4                 
                 ]#, *section_names_slide3 ,
path = os.getcwd() + '\Landscape base.pptx'
new_pre = os.getcwd() + '\Landscape duplicate Market(Trends).pptx'


In [52]:
print(len(index))
print(index)
print(len(duplication))
print(duplication)
print(len(section_names))
print(section_names)


11
[0, 0, 0, 0, 1, 1, 1, 1, 3, 3, 3]
11
[21, 21, 3, 3, 21, 21, 3, 3, 12, 12, 22]
11
['Market Trends by Manufacturer', 'Market Trends by Brands', 'Market Trends by Sectors', 'Market Trends by Segments', 'Market Concentration By Manufacturer', 'Market Concentration By Brands', 'Market Concentration By Sectors', 'Market Concentration By Segments', 'Value Vs AvgPrice By Sectors', 'Value Vs AvgPrice By Segments', 'Value Vs AvgPrice By Retailer For Region']


In [53]:
print(sum(duplication))

142


In [54]:
slideDuplication(index,duplication,section_names,path,new_pre)

In [55]:
prs = Presentation(new_pre)

In [56]:
def calculate_position(end):
    return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))

#### Slide 1_ Market_Trends

In [57]:
p=0
Market_Trends(prs, list(modified_manuf_dfs_new.keys()), modified_manuf_dfs_new, modified_manuf_totals_new, client_manuf ,position = calculate_position(p), slide_by = f'{ManufOrTopC}', years=years)
p+=1

Market_Trends(prs, list(modified_brands_share_new.keys()), modified_brands_share_new, modified_brands_totals_new, client_brands ,position =calculate_position(p), slide_by = f'{BrandOrTopB}', years=years)
p+=1

if len(sectors)!=0:
    Market_Trends(prs, list(modified_sectors_dfs_new.keys()), modified_sectors_dfs_new, sectors_totals_new, sectors ,position = calculate_position(p), slide_by = 'Sector', years=years)
    p+=1

if len(segments)!=0:
    Market_Trends(prs, list(modified_segment_dfs_new.keys()), modified_segment_dfs_new, segment_totals_new, segments ,position = calculate_position(p), slide_by = 'Segment', years=years)
    p+=1

    
if len(subsegments)!=0:
    Market_Trends(prs, list(modified_subsegment_dfs_new.keys()), modified_subsegment_dfs_new, subsegment_totals_new, subsegments ,position = calculate_position(p), slide_by = 'SubSegment', years=years)
    p+=1

if len(subcategories)!=0:
    Market_Trends(prs, list(modified_subcategories_dfs_new.keys()), modified_subcategories_dfs_new, subcategories_totals_new, subcategories ,position = calculate_position(p), slide_by = 'SubCategory', years=years)
    p+=1


In [58]:
for k in modified_brands_share_new.keys():
    modified_brands_share_new[k][f'{BrandOrTopB}'] = modified_brands_share_new[k][f'{BrandOrTopB}'].str.replace("Private Label", "PL")

#### Slide 2_Market_Concentration

In [59]:
Market_Concentration(prs,list(modified_manuf_dfs_new.keys()), modified_manuf_dfs_new, modified_manuf_totals_new, client_manuf ,position =  calculate_position(p), slide_by = f'{ManufOrTopC}',years=years)
p+=1
Market_Concentration(prs,list(modified_brands_share_new.keys()), modified_brands_share_new, modified_brands_totals_new, client_brands ,position =  calculate_position(p), slide_by = f'{BrandOrTopB}',years=years)
p+=1
if len(sectors)!=0:
    Market_Concentration(prs,list(modified_sectors_dfs_new.keys()), modified_sectors_dfs_new, sectors_totals_new, sectors ,position =  calculate_position(p), slide_by = 'Sector',years=years)
    p+=1
if len(segments)!=0:
    Market_Concentration(prs,list(modified_segment_dfs_new.keys()), modified_segment_dfs_new, segment_totals_new, segments ,position =  calculate_position(p), slide_by = 'Segment',years=years)
    p+=1

if len(subsegments)!=0:
    Market_Concentration(prs,list(modified_subsegment_dfs_new.keys()), modified_subsegment_dfs_new, subsegment_totals_new, subsegments ,position =  calculate_position(p), slide_by = 'SubSegment',years=years)
    p+=1

if len(subcategories)!=0:
    Market_Concentration(prs,list(modified_subcategories_dfs_new.keys()), modified_subcategories_dfs_new, subcategories_totals_new, subcategories ,position =  calculate_position(p), slide_by = 'SubCategory',years=years)
    p+=1


#### Slide 3_Market_Growth_Contribution

#### Slide4_ValueSales_AvgPrice

In [60]:
market_list=["NATIONAL"] + regions_RET + channels_RET+market_RET +  regions_CHAN +channels_CHAN+ market_CHAN+regions_REG+channels_REG+market_REG+regions_CUST +channels_CUST+ market_CUST
manuf_brand_list = client_manuf + client_brands
if len(sectors):
    ValueSales_AvgPrice(prs, list(modified_sectors_clients_new.keys()), globals()[f"modified_sectors_{slides_Period}_new"], modified_sectors_clients_new,manuf_brand_list, market_list,position = calculate_position(p), slide_by = 'Sector')
    ValueSales_AvgPrice(prs, list(modified_sectors_clients_new.keys()), globals()[f"modified_sectors_{slides_Period}_new"], modified_sectors_clients_new,manuf_brand_list, market_list,position = calculate_position(p), slide_by = 'Sector')
    p+=1

if len(segments)!=0:
    ValueSales_AvgPrice(prs, list(modified_segment_clients_new.keys()), globals()[f"modified_segment_{slides_Period}_new"], modified_segment_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'Segment')
    ValueSales_AvgPrice(prs, list(modified_segment_clients_new.keys()), globals()[f"modified_segment_{slides_Period}_new"], modified_segment_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'Segment')
    p+=1

if len(subsegments)!=0:
    ValueSales_AvgPrice(prs, list(modified_subsegment_clients_new.keys()), globals()[f"modified_subsegment_{slides_Period}_new"], modified_subsegment_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'SubSegment')
    ValueSales_AvgPrice(prs, list(modified_subsegment_clients_new.keys()), globals()[f"modified_subsegment_{slides_Period}_new"], modified_subsegment_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'SubSegment')
    p+=1

if len(subcategories)!=0:
    ValueSales_AvgPrice(prs, list(modified_subcategories_clients_new.keys()), globals()[f"modified_subcategories_{slides_Period}_new"], modified_subcategories_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'SubCategory')
    ValueSales_AvgPrice(prs, list(modified_subcategories_clients_new.keys()), globals()[f"modified_subcategories_{slides_Period}_new"], modified_subcategories_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'SubCategory')
    p+=1


In [61]:
catg_list=[f'{categories[0]}'] + sectors + segments + subsegments + subcategories
for key,val in market_list_Avg.items():
    ValueSales_AvgPrice(prs, list(val[2].keys()), val[1], val[2],manuf_brand_list,catg_list ,position = calculate_position(p), slide_by = val[3], slide_for = val[4])
    ValueSales_AvgPrice(prs, list(val[2].keys()), val[1], val[2],manuf_brand_list,catg_list ,position = calculate_position(p), slide_by = val[3], slide_for = val[4])
    p+=1

In [62]:
outputPath = os.getcwd()+f"\\Final Output\\Landscape Market(trend-Concent-growth)&ValAvgPrice_{datetime.today().strftime("%d-%m")}.pptx"
prs.save(outputPath)

## Second Landscape Deck: Category Trend - Overview, Share Evolution

In [63]:
index = [8, 9,9,10]
index=[i for i in index if i !=[]]


duplication_7 = [len(modified_calendar_new.keys())]
duplication_8_bymanuf = [len(modified_manuf_evolution_sorted_new.keys())]
duplication_8 = [len(modified_brands_evolution_sorted_new.keys())]
duplication_9 = [len(modified_categories_dfs_new)]

duplication = duplication_7 + duplication_8_bymanuf+duplication_8 + duplication_9
duplication = [item for item in duplication if item !=0]


section_names_slide7 =  ["Category Trends"]
section_names_slide8_bymanuf =  ['Share Evolution By Manuf']
section_names_slide8 =  ['Share Evolution By Brand']
section_names_slide9 =  ["Category Overview"]

section_names = [*section_names_slide7,*section_names_slide8_bymanuf,*section_names_slide8,*section_names_slide9]

path = os.getcwd() + '\Landscape base.pptx'
new_pre = os.getcwd() + '\Landscape duplicate Category(Trend-Overview).pptx'



In [64]:
index = [8, 9,9,10]
index=[i for i in index if i !=[]]


duplication_7 = [len(modified_calendar_new.keys())]
duplication_8_bymanuf = [len(modified_manuf_evolution_sorted_new.keys())]
duplication_8 = [len(modified_brands_evolution_sorted_new.keys())]
duplication_9 = [len(modified_categories_dfs_new)]

duplication = duplication_7 + duplication_8_bymanuf+duplication_8 + duplication_9
duplication = [item for item in duplication if item !=0]


section_names_slide7 =  ["Category Trends"]
section_names_slide8_bymanuf =  ['Share Evolution By Manuf']
section_names_slide8 =  ['Share Evolution By Brand']
section_names_slide9 =  ["Category Overview"]

section_names = [*section_names_slide7,*section_names_slide8_bymanuf,*section_names_slide8,*section_names_slide9]

path = os.getcwd() + '\Landscape base.pptx'
new_pre = os.getcwd() + '\Landscape duplicate Category(Trend-Overview).pptx'



In [65]:
print(len(index))
print(index)
print(len(duplication))
print(duplication)
print(len(section_names))
print(section_names)

print(sum(duplication))


4
[8, 9, 9, 10]
4
[30, 21, 21, 3]
4
['Category Trends', 'Share Evolution By Manuf', 'Share Evolution By Brand', 'Category Overview']
75


In [66]:
slideDuplication(index,duplication,section_names,path,new_pre)

In [67]:
prs = Presentation(new_pre)

def calculate_position(end):
    return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))

#### Slide9_Category_Trend

In [68]:
p = 0
Category_Trend(prs,list(modified_calendar_new.keys()), modified_calendar_new, position = calculate_position(p))
p+=1

#### Slide10_Share_Evolution

In [69]:
for k in modified_brands_evolution_sorted_new.keys():
    t = modified_brands_evolution_sorted_new[k].copy()
    t['Year'] = t['Year'].astype(str)
    mask = t['Year'].str.contains('Total', na=False)
    t.loc[mask, f'{BrandOrTopB}'] = 'Total'
    t.loc[mask, 'Year'] = t.loc[mask, 'Year'].str.replace(' Total', '', regex=True)
    t['Tot Sort'] = np.where(t[f'{BrandOrTopB}'] == 'Total', 0, 1)
    t = t.sort_values(['Year', 'Tot Sort'], ascending=[True, True])
    t = t.drop(columns='Tot Sort')
    modified_brands_evolution_sorted_new[k] = t

for k in modified_manuf_evolution_sorted_new.keys():
    t = modified_manuf_evolution_sorted_new[k].copy()
    t['Year'] = t['Year'].astype(str)  
    mask = t['Year'].str.contains('Total', na=False)
    t.loc[mask, f'{ManufOrTopC}'] = 'Total'
    t.loc[mask, 'Year'] = t.loc[mask, 'Year'].str.replace(' Total', '', regex=True)
    t['Tot Sort'] = np.where(t[f'{ManufOrTopC}'] == 'Total', 0, 1)
    t = t.sort_values(['Year', 'Tot Sort'], ascending=[True, True])
    t = t.drop(columns='Tot Sort')
    modified_manuf_evolution_sorted_new[k] = t

In [70]:
Share_Evolution(prs, list(modified_manuf_evolution_sorted_new.keys()), modified_manuf_evolution_sorted_new, position = calculate_position(p),colname=f"{ManufOrTopC}")
p+=1
Share_Evolution(prs, list(modified_brands_evolution_sorted_new.keys()), modified_brands_evolution_sorted_new, position = calculate_position(p),colname=f'{BrandOrTopB}')
p+=1

### Slide11_CategoryOverview

In [71]:
CategoryOverview(prs,modified_categories_dfs_new,col='MonthYear',position=calculate_position(p),scope = sectors, slide_by = 'Sector')
p+=1

In [72]:
outputPath = os.getcwd()+f"\\Final Output\\Landscape Category(Trend-Overview)_{datetime.today().strftime("%d-%m")}.pptx"
prs.save(outputPath)

## Third Landscape Deck (Share and Growth)

In [73]:
categorizations = [
    ("Category", categories),
    ("Sector", sectors),
    ("Segment", segments),
    ("SubSegment", subsegments),
    ("SubCategory", subcategories),

]
markets_val = [regions_RET , channels_RET,market_RET  ,regions_CHAN ,channels_CHAN, market_CHAN ,regions_REG,channels_REG,market_REG,regions_CUST ,channels_CUST, market_CUST]

# Initialize the updated dictionaries and lists
market_Catg_list = {}
# slide13Dup = []
section_names_slide_13 = []

# Loop through the original markets
for idx, val in enumerate(markets_val):
    if len(val) > 0:
        for cat_name, cat_list in categorizations:
            if len(cat_list) > 0:
                # Modify key and section names to include the categorization level
                key_name = f"{markets_name[idx]}_{cat_name}"
                section_title = f"Share and Growth By {cat_name} for {slide_by[idx]} For {slide_for[idx]}"
                
                # Add the categorized data to the dictionary
                market_Catg_list[key_name] = [val, markets_dfs[idx], client_dfs[idx], slide_by[idx], slide_for[idx], cat_list]
                # slide13Dup.append(len(client_dfs[idx].keys()))
                section_names_slide_13.append(section_title)

In [74]:
retailerDup = [len(retailerCatDic),len(retailerSecDic),len(retailerSegDic),len(retailerSubSegDic),len(retailerSubCatDic)]*retMarketLength
channelDup = [len(channelCatDic),len(channelSecDic),len(channelSegDic),len(channelSubSegDic),len(channelSubCatDic)]*chanMarketLength
regionDup = [len(regionCatDic),len(regionSecDic),len(regionSegDic),len(regionSubSegDic),len(regionSubCatDic)]*regMarketLength
custDup = [len(custCatDic),len(custSecDic),len(custSegDic),len(custSubSegDic),len(custSubCatDic)]*custMarketLength


In [75]:
slide13Index=[*[12]* (((5 -((not sectors) + (not segments) + (not subsegments)+ (not subcategories)) )) *(12 - ((not regions_RET)+(not channels_RET) + (not market_RET)+ (not regions_CHAN)+ (not channels_CHAN)+ (not market_CHAN)+ (not regions_REG)+ (not channels_REG)+ (not market_REG)+ (not regions_CUST)+ (not channels_CUST)+ (not market_CUST))  ))]

slide13Dup = retailerDup + channelDup+regionDup + custDup
slide_12_13_index = [*[11]*(4 -((not sectors) + (not segments) + (not subsegments)+ (not subcategories)) ) ]+slide13Index

duplication = [len(cea_sectors_clients_new.keys()),len(cea_segment_clients_new.keys()),len(cea_subsegment_clients_new.keys()),len(cea_subcategory_clients_new.keys())] + slide13Dup
duplication_12_13 =[ i for i in duplication if i !=0 ]
section_names_slide11 =  (["Sales and Growth By Sector"] if sectors else [])+(["Sales and Growth By Segment"] if len(segments)>0 else [])+(["Sales and Growth By SubSegment"] if len(subsegments)>0 else [])+(["Sales and Growth By SubCategories"] if len(subcategories)>0 else [])

section_names_12_13 = [*section_names_slide11,*section_names_slide_13]

In [76]:
print(len(slide_12_13_index))
print(slide_12_13_index)
print(len(duplication_12_13))
print(duplication_12_13)
print(len(section_names_12_13))
print(section_names_12_13)

5
[11, 11, 12, 12, 12]
5
[12, 12, 4, 9, 9]
5
['Sales and Growth By Sector', 'Sales and Growth By Segment', 'Share and Growth By Category for Region For Retailer', 'Share and Growth By Sector for Region For Retailer', 'Share and Growth By Segment for Region For Retailer']


In [77]:
momentum_index=[*[13]* (1+((4 -((not segments) + (not subsegments)+ (not subcategories)) )) *(4 - ((not retailers)+(not channels)+(not regions) + (not cust))  ))]

momentum_index=[i for i in momentum_index if i !=[]]

momentum_dupl_1 = [(len(modified_momentum_retailers.keys())if retailers else 0), (len(modified_momentum_channels.keys())if channels else 0), (len(modified_momentum_regions.keys())if regions else 0), (len(modified_momentum_cust.keys())if cust else 0)]
momentum_dupl_2 = [(len(modified_momentum_retailers_seg.keys())if retailers else 0), (len(modified_momentum_channels_seg.keys())if channels else 0), (len(modified_momentum_regions_seg.keys())if regions else 0), (len(modified_momentum_cust_seg.keys())if cust else 0)]if segments else [0]
momentum_dupl_3 = [(len(modified_momentum_retailers_subseg.keys())if retailers else 0), (len(modified_momentum_channels_subseg.keys())if channels else 0), (len(modified_momentum_regions_subseg.keys())if regions else 0), (len(modified_momentum_cust_subseg.keys())if cust else 0)]if subsegments else [0]
momentum_dupl_4 = [(len(modified_momentum_retailers_subcatg.keys())if retailers else 0), (len(modified_momentum_channels_subcatg.keys())if channels else 0), (len(modified_momentum_regions_subcatg.keys())if regions else 0), (len(modified_momentum_cust_subcatg.keys())if cust else 0)]if subcategories else [0]
momentum_dupl_5 = [len(modified_momentum_dfs.keys())]


momentum_duplication = momentum_dupl_1 + momentum_dupl_2 + momentum_dupl_3 + momentum_dupl_4 + momentum_dupl_5 
momentum_duplication = [item for item in momentum_duplication if item !=0]

momentum_section_name1 = (['Momentum Analysis By Retailers'] if len(retailers)>0 else [] ) + ([ 'Momentum Analysis By Channels'] if len(channels)>0 else [] )+([ 'Momentum Analysis By Regions'] if len(regions)>0 else [] ) + ([ f'Momentum Analysis By {customareas}'] if len(cust)>0 else [])
momentum_section_name2 = ((['Momentum Analysis By Retailers By Segments'] if len(retailers)>0 else [] ) + ([ 'Momentum Analysis By Channels By Segments'] if len(channels)>0 else [] )+ ([ 'Momentum Analysis By Regions By Segments'] if len(regions)>0 else [] ) + ([ f'Momentum Analysis By {customareas} By Segments'] if len(cust)>0 else [] ))if segments else []
momentum_section_name3 = ((['Momentum Analysis By Retailers By SubSegments'] if len(retailers)>0 else [] ) + ([ 'Momentum Analysis By Channels By SubSegments'] if len(channels)>0 else [] ) + ([ 'Momentum Analysis By Regions By SubSegments'] if len(regions)>0 else [] ) + ([ f'Momentum Analysis By {customareas} By SubSegments'] if len(cust)>0 else [] ))if subsegments else []
momentum_section_name4 = ((['Momentum Analysis By Retailers By SubCategories'] if len(retailers)>0 else [] ) + ([ 'Momentum Analysis By Channels By SubCategories'] if len(channels)>0 else [] ) + ([ 'Momentum Analysis By Regions By SubCategories'] if len(regions)>0 else [] ) + ([ f'Momentum Analysis By {customareas} By SubCategories'] if len(cust)>0 else [] ))if subcategories else []
momentum_section_name5 = "Momentum Analysis By Sectors"

momentum_section_names = [ *momentum_section_name1,*momentum_section_name2,
                  *momentum_section_name3, *momentum_section_name4, momentum_section_name5  
                 ]


In [78]:
index=[
     4,
                retailerREGIndex if regions_RET else [], retailerREGIndex if regions_RET else [], 
                retailerCHAIndex if channels_RET else [], retailerCHAIndex if channels_RET else [], 
                retailerMARIndex if market_RET else [], retailerMARIndex if market_RET else [], 
                channelREGIndex if regions_CHAN else [], channelREGIndex if regions_CHAN else [], 
                channelCHAIndex if channels_CHAN else [], channelCHAIndex if channels_CHAN else [], 
                channelMARIndex if market_CHAN else [], channelMARIndex if market_CHAN else [], 
                regionREGIndex if regions_REG else [], regionREGIndex if regions_REG else [], 
                regionCHAIndex if channels_REG else [], regionCHAIndex if channels_REG else [], 
                regionMARIndex if market_REG else [], regionMARIndex if market_REG else [], 
                custREGIndex if regions_CUST else [], custREGIndex if regions_CUST else [], 
                custCHAIndex if channels_CUST else [], custCHAIndex if channels_CUST else [],  
                custMARIndex if market_CUST else [], custMARIndex if market_CUST else [],           
                sectorIndex if sectors else [],sectorIndex if sectors else [],
                segmentIndex if segments else [],segmentIndex if segments else [],
                subsegmentIndex if subsegments else [],subsegmentIndex if subsegments else [],
                subcategoryIndex if subcategories else [],subcategoryIndex if subcategories else []] + slide_12_13_index + momentum_index
index=[i for i in index if i !=[]]

duplication_5 = [len(globals()[f"modified_manuf_{slides_Period}_new"].keys())]
duplication_6area = [1 if regions_RET else 0, 1 if regions_RET else 0,1 if channels_RET else 0, 1 if channels_RET else 0,1 if market_RET else 0, 1 if market_RET else 0,1 if regions_CHAN else 0, 1 if regions_CHAN else 0,1 if channels_CHAN else 0, 1 if channels_CHAN else 0,1 if market_CHAN else 0, 1 if market_CHAN else 0,1 if regions_REG else 0, 1 if regions_REG else 0, 1 if channels_REG else 0, 1 if channels_REG else 0, 1 if market_REG else 0, 1 if market_REG else 0,1 if regions_CUST else 0, 1 if regions_CUST else 0, 1 if channels_CUST else 0, 1 if channels_CUST else 0, 1 if market_CUST else 0, 1 if market_CUST else 0]
duplication_6 = [1 if sectors else 0, 1 if sectors else 0, 1 if segments else 0, 1 if segments else 0, 1 if subsegments else 0, 1 if subsegments else 0, 1 if subcategories else 0, 1 if subcategories else 0]

duplication = duplication_5 + duplication_6area + duplication_6 + duplication_12_13 + momentum_duplication

duplication = [item for item in duplication if item !=0]

section_names_slide5 = ['Share and Growth By Manufacturer/Brands']
section_names_slide6area = (['Share And Growth By Manufacturer By Retailer', 'Share And Growth By Brands By Retailer'] if len(regions_RET)>0 else [] ) +(['Share And Growth By Manufacturer By Retailer', 'Share And Growth By Brands By Retailer'] if len(channels_RET)>0 else [] ) +(['Share And Growth By Manufacturer By Retailer', 'Share And Growth By Brands By Retailer'] if len(market_RET)>0 else [] ) + (['Share And Growth By Manufacturer By Channel', 'Share And Growth By Brands By Channel'] if len(regions_CHAN)>0 else [] ) + (['Share And Growth By Manufacturer By Channel', 'Share And Growth By Brands By Channel'] if len(channels_CHAN)>0 else [] ) + (['Share And Growth By Manufacturer By Channel', 'Share And Growth By Brands By Channel'] if len(market_CHAN)>0 else [] ) +(['Share And Growth By Manufacturer By Region', 'Share And Growth By Brands By Region'] if len(regions_REG)>0 else [] ) + (['Share And Growth By Manufacturer By Region', 'Share And Growth By Brands By Region'] if len(channels_REG)>0 else [] ) + (['Share And Growth By Manufacturer By Region', 'Share And Growth By Brands By Region'] if len(market_REG)>0 else [] ) + (['Share And Growth By Manufacturer By '+ customareas, 'Share And Growth By Brands By '+ customareas] if len(regions_CUST)>0 else [] ) + (['Share And Growth By Manufacturer By '+ customareas, 'Share And Growth By Brands By '+ customareas] if len(channels_CUST)>0 else [] ) + (['Share And Growth By Manufacturer By '+ customareas, 'Share And Growth By Brands By '+ customareas] if len(market_CUST)>0 else [] )
section_names_slide6 = (['Share And Growth By Manufacturer By Sector', 'Share And Growth By Brands By Sector'] if len(sectors)>0 else [] ) + (['Share And Growth By Manufacturer By Segment', 'Share And Growth By Brands By Segment'] if len(segments)>0 else [] ) + (['Share And Growth By Manufacturer By SubSegment', 'Share And Growth By Brands By SubSegment'] if len(subsegments)>0 else [] )  + (['Share And Growth By Manufacturer By SubCategory', 'Share And Growth By Brands By SubCategory'] if len(subcategories)>0 else [] )


section_names=[*section_names_slide5,*section_names_slide6area,*section_names_slide6] + section_names_12_13 + momentum_section_names
path = os.getcwd() + '\Landscape base.pptx'
new_pre = os.getcwd() + '\Landscape duplicate ShareGrowth.pptx'


In [79]:
print(len(index))
print(index)
print(len(duplication))
print(duplication)
print(len(section_names))
print(section_names)
print(sum(duplication))

15
[4, [6, 6, 6, 6, 6, 6, 6], [6, 6, 6, 6, 6, 6, 6], [7, 7, 7], [7, 7, 7], [7, 7, 7], [7, 7, 7], 11, 11, 12, 12, 12, 13, 13, 13]
15
[21, 1, 1, 1, 1, 1, 1, 12, 12, 4, 9, 9, 7, 1, 3]
15
['Share and Growth By Manufacturer/Brands', 'Share And Growth By Manufacturer By Retailer', 'Share And Growth By Brands By Retailer', 'Share And Growth By Manufacturer By Sector', 'Share And Growth By Brands By Sector', 'Share And Growth By Manufacturer By Segment', 'Share And Growth By Brands By Segment', 'Sales and Growth By Sector', 'Sales and Growth By Segment', 'Share and Growth By Category for Region For Retailer', 'Share and Growth By Sector for Region For Retailer', 'Share and Growth By Segment for Region For Retailer', 'Momentum Analysis By Retailers', 'Momentum Analysis By Retailers By Segments', 'Momentum Analysis By Sectors']
84


In [80]:
slideDuplication(index,duplication,section_names,path,new_pre)

In [81]:
prs = Presentation(new_pre)

def calculate_position(end):
    return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))

#### Slide5_Share_Growth

In [82]:
for k in globals()[f"modified_brands_{slides_Period}_new"].keys():
    t=globals()[f"modified_brands_{slides_Period}_new"][k].copy()
    t= t.rename(columns = { f"Value Share {slides_Period}":"Value Share", f"Share DYA {slides_Period}":"Share DYA", f"Av Price/KG {slides_Period}":"Av Price/KG"})
    globals()[f"modified_brands_{slides_Period}_new"][k] = t

In [83]:
p = 0
Share_Growth(prs, list(globals()[f"modified_manuf_{slides_Period}_new"].keys()), globals()[f"modified_manuf_{slides_Period}_new"], globals()[f"modified_brands_{slides_Period}_new"], position =calculate_position(p))
p+=1

In [84]:
for k in globals()[f"modified_brands_{slides_Period}_new"].keys():
    t=globals()[f"modified_brands_{slides_Period}_new"][k].copy()
    t= t.rename(columns = {"Value Share": f"Value Share {slides_Period}", "Share DYA":f"Share DYA {slides_Period}", "Av Price/KG":f"Av Price/KG {slides_Period}"})
    globals()[f"modified_brands_{slides_Period}_new"][k] = t[t[f'Value Share {slides_Period}'] != 0]
    


### Slide6_Share and Growth By Manufacturer 

In [85]:
c = calculate_position(p)
if len(regions_RET)!=0:
    for key in shareGrowthDf_retailers_REG.keys():
        retailers_REG_list = [shareGrowthDf_retailers_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_REG[key]), 3)]
        for sublist in retailers_REG_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_manuf_{slides_Period}_new"],position=c,slide_by='Manufacturer | By Retailer', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_retailers_REG.keys():
        retailers_REG_list = [shareGrowthDf_retailers_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_REG[key]), 3)]
        for sublist in retailers_REG_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_brands_{slides_Period}_new"],position=c,slide_by='Brands | By Retailer', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1
    p+=1 
if len(channels_RET)!=0:
    c=calculate_position(p) 
    for key in shareGrowthDf_retailers_CHAN.keys():
        retailers_CHAN_list = [shareGrowthDf_retailers_CHAN[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_CHAN[key]), 3)]
        for sublist in retailers_CHAN_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_manuf_{slides_Period}_new"],position=c,slide_by='Manufacturer | By Retailer', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_retailers_CHAN.keys():
        retailers_CHAN_list = [shareGrowthDf_retailers_CHAN[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_CHAN[key]), 3)]
        for sublist in retailers_CHAN_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_brands_{slides_Period}_new"],position=c,slide_by='Brands | By Retailer', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  
    p+=1
if len(market_RET)!=0:
    c=calculate_position(p) 
    for key in shareGrowthDf_retailers_MAR.keys():
        retailers_MAR_list = [shareGrowthDf_retailers_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_MAR[key]), 3)]
        for sublist in retailers_MAR_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_manuf_{slides_Period}_new"],position=c,slide_by='Manufacturer | By Retailer', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_retailers_MAR.keys():
        retailers_MAR_list = [shareGrowthDf_retailers_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_MAR[key]), 3)]
        for sublist in retailers_MAR_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_brands_{slides_Period}_new"],position=c,slide_by='Brands | By Retailer', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1 
    p+=1
    

Total Fromage
Soft Cheese
Aperitif
Ingredient A Chaud
Enfant
Frais A Tartiner
Salade
Total Fromage
Soft Cheese
Aperitif
Ingredient A Chaud
Enfant
Frais A Tartiner
Salade


In [86]:
c=calculate_position(p)  
if len(regions_CHAN)!=0:
    for key in shareGrowthDf_channels_REG.keys():
        channels_REG_list = [shareGrowthDf_channels_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_channels_REG[key]), 3)]
        for sublist in channels_REG_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_manuf_{slides_Period}_new"],position=c,slide_by='Manufacturer | By Channel', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_channels_REG.keys():
        channels_REG_list = [shareGrowthDf_channels_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_channels_REG[key]), 3)]
        for sublist in channels_REG_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_brands_{slides_Period}_new"],position=c,slide_by='Brands | By Channel', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1
    p+=1   
if len(channels_CHAN)!=0:  
    c=calculate_position(p) 
    for key in shareGrowthDf_channels_CHAN.keys():
        channels_CHAN_list = [shareGrowthDf_channels_CHAN[key][i:i+3] for i in range(0, len(shareGrowthDf_channels_CHAN[key]), 3)]
        for sublist in channels_CHAN_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_manuf_{slides_Period}_new"],position=c,slide_by='Manufacturer | By Channel', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_channels_CHAN.keys():
        channels_CHAN_list = [shareGrowthDf_channels_CHAN[key][i:i+3] for i in range(0,len(shareGrowthDf_channels_CHAN[key]), 3)]
        for sublist in channels_CHAN_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_brands_{slides_Period}_new"],position=c,slide_by='Brands | By Channel', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  
    p+=1
if len(market_CHAN)!=0:
    c=calculate_position(p) 
    for key in shareGrowthDf_channels_MAR.keys():
        channels_MAR_list = [shareGrowthDf_channels_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_channels_MAR[key]), 3)]
        for sublist in channels_MAR_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_manuf_{slides_Period}_new"],position=c,slide_by='Manufacturer | By Channel', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_channels_MAR.keys():
        channels_MAR_list = [shareGrowthDf_channels_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_channels_MAR[key]), 3)]
        for sublist in channels_MAR_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_brands_{slides_Period}_new"],position=c,slide_by='Brands | By Channel', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1 
    p+=1

In [87]:
c = calculate_position(p)
if len(regions_REG)!=0:
    for key in shareGrowthDf_regions_REG.keys():
        regions_REG_list = [shareGrowthDf_regions_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_regions_REG[key]), 3)]
        for sublist in regions_REG_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_manuf_{slides_Period}_new"],position=c,slide_by='Manufacturer | By Region', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_regions_REG.keys():
        regions_REG_list = [shareGrowthDf_regions_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_regions_REG[key]), 3)]
        for sublist in regions_REG_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_brands_{slides_Period}_new"],position=c,slide_by='Brands | By Region', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1
    p+=1 
if len(channels_REG)!=0:
    c=calculate_position(p) 
    for key in shareGrowthDf_regions_CHAN.keys():
        regions_CHAN_list = [shareGrowthDf_regions_CHAN[key][i:i+3] for i in range(0, len(shareGrowthDf_regions_CHAN[key]), 3)]
        for sublist in regions_CHAN_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_manuf_{slides_Period}_new"],position=c,slide_by='Manufacturer | By Region', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_regions_CHAN.keys():
        regions_CHAN_list = [shareGrowthDf_regions_CHAN[key][i:i+3] for i in range(0, len(shareGrowthDf_regions_CHAN[key]), 3)]
        for sublist in regions_CHAN_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_brands_{slides_Period}_new"],position=c,slide_by='Brands | By Region', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  
    p+=1
if len(market_REG)!=0:
    c=calculate_position(p) 
    for key in shareGrowthDf_regions_MAR.keys():
        regions_MAR_list = [shareGrowthDf_regions_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_regions_MAR[key]), 3)]
        for sublist in regions_MAR_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_manuf_{slides_Period}_new"],position=c,slide_by='Manufacturer | By Region', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_regions_MAR.keys():
        regions_MAR_list = [shareGrowthDf_regions_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_regions_MAR[key]), 3)]
        for sublist in regions_MAR_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_brands_{slides_Period}_new"],position=c,slide_by='Brands | By Region', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1 
    p+=1
    

In [88]:
c=calculate_position(p)  
if len(regions_CUST)!=0:
    for key in shareGrowthDf_custom_REG.keys():
        custom_REG_list = [shareGrowthDf_custom_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_REG[key]), 3)]
        for sublist in custom_REG_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_manuf_{slides_Period}_new"],position=c,slide_by='Manufacturer | By '+ customareas, list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_custom_REG.keys():
        custom_REG_list = [shareGrowthDf_custom_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_REG[key]), 3)]
        for sublist in custom_REG_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_brands_{slides_Period}_new"],position=c,slide_by='Brands | By '+ customareas, list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1
    p+=1     
if len(channels_CUST)!=0:
    c=calculate_position(p) 
    for key in shareGrowthDf_custom_CHAN.keys():
        custom_CHAN_list = [shareGrowthDf_custom_CHAN[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_CHAN[key]), 3)]
        for sublist in custom_CHAN_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_manuf_{slides_Period}_new"],position=c,slide_by='Manufacturer | By '+ customareas, list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_custom_CHAN.keys():
        custom_CHAN_list = [shareGrowthDf_custom_CHAN[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_CHAN[key]), 3)]
        for sublist in custom_CHAN_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_brands_{slides_Period}_new"],position=c,slide_by='Brands | By '+ customareas, list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  
    p+=1
if len(market_CUST)!=0:
    c=calculate_position(p) 
    for key in shareGrowthDf_custom_MAR.keys():
        custom_MAR_list = [shareGrowthDf_custom_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_MAR[key]), 3)]
        for sublist in custom_MAR_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_manuf_{slides_Period}_new"],position=c,slide_by='Manufacturer | By '+ customareas, list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_custom_MAR.keys():
        custom_MAR_list = [shareGrowthDf_custom_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_MAR[key]), 3)]
        for sublist in custom_MAR_list:
            Share_Growth_Separately_area( prs,[key], globals()[f"modified_brands_{slides_Period}_new"],position=c,slide_by='Brands | By '+ customareas, list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1 
    p+=1

In [89]:
c = calculate_position(p)
if len(sectors)!=0:
    for key in sharGrowthDf_sec.keys():
        sectors_list = [sharGrowthDf_sec[key][i:i+3] for i in range(0, len(sharGrowthDf_sec[key]), 3)]
        print(sectors_list)
        for sublist in sectors_list:
            print(sublist)
            Share_Growth_Separately( prs,[key], globals()[f"modified_manuf_{slides_Period}_new"],position=c,slide_by='Manufacturer | By Sector', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1

    c=calculate_position(p)
    for key in sharGrowthDf_sec.keys():
        sectors_list = [sharGrowthDf_sec[key][i:i+3] for i in range(0, len(sharGrowthDf_sec[key]), 3)]
        for sublist in sectors_list:
            Share_Growth_Separately( prs,[key], globals()[f"modified_brands_{slides_Period}_new"],position=c,slide_by='Brands | By Sector', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  
    p+=1
if len(segments)!=0:
    c = calculate_position(p)
    for key in sharGrowthDf_seg.keys():
        segments_list = [sharGrowthDf_seg[key][i:i+3] for i in range(0, len(sharGrowthDf_seg[key]), 3)]
        for sublist in segments_list:
            Share_Growth_Separately( prs,[key], globals()[f"modified_manuf_{slides_Period}_new"],position=c,slide_by='Manufacturer | By Segment', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    
    c=calculate_position(p)
    for key in sharGrowthDf_seg.keys():
        segments_list = [sharGrowthDf_seg[key][i:i+3] for i in range(0, len(sharGrowthDf_seg[key]), 3)]
        for sublist in segments_list:
            Share_Growth_Separately( prs,[key], globals()[f"modified_brands_{slides_Period}_new"],position=c,slide_by='Brands | By Segment', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1

if len(subsegments)!=0:
    c = calculate_position(p)
    for key in sharGrowthDf_subseg.keys():
        subsegments_list = [sharGrowthDf_subseg[key][i:i+3] for i in range(0, len(sharGrowthDf_subseg[key]), 3)]
        for sublist in subsegments_list:
            Share_Growth_Separately( prs,[key], globals()[f"modified_manuf_{slides_Period}_new"],position=c,slide_by='Manufacturer | By SubSegment', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  # Correctly incrementing the variable c
    p+=1
    
    c=calculate_position(p)
    for key in sharGrowthDf_subseg.keys():
        subsegments_list = [sharGrowthDf_subseg[key][i:i+3] for i in range(0, len(sharGrowthDf_subseg[key]), 3)]
        for sublist in subsegments_list:
            Share_Growth_Separately( prs,[key], globals()[f"modified_brands_{slides_Period}_new"],position=c,slide_by='Brands | By SubSegment', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1


if len(subcategories)!=0:
    c = calculate_position(p)
    for key in sharGrowthDf_subcat.keys():
        subcateg_list = [sharGrowthDf_subcat[key][i:i+3] for i in range(0, len(sharGrowthDf_subcat[key]), 3)]
        for sublist in subcateg_list:
            Share_Growth_Separately( prs,[key], globals()[f"modified_manuf_{slides_Period}_new"],position=c,slide_by='Manufacturer | By SubCategory', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  # Correctly incrementing the variable c
    p+=1
    
    c=calculate_position(p)
    for key in sharGrowthDf_subcat.keys():
        subcateg_list = [sharGrowthDf_subcat[key][i:i+3] for i in range(0, len(sharGrowthDf_subcat[key]), 3)]
        for sublist in subcateg_list:
            Share_Growth_Separately( prs,[key], globals()[f"modified_brands_{slides_Period}_new"],position=c,slide_by='Brands | By SubCategory', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1            

[['Soft Cheese', 'Aperitif', 'Ingredient A Chaud']]
['Soft Cheese', 'Aperitif', 'Ingredient A Chaud']
NATIONAL
[['Soft Cheese', 'Aperitif', 'Ingredient A Chaud']]
['Soft Cheese', 'Aperitif', 'Ingredient A Chaud']
Carrefour
[['Soft Cheese', 'Aperitif', 'Ingredient A Chaud']]
['Soft Cheese', 'Aperitif', 'Ingredient A Chaud']
Intermarche
NATIONAL
Carrefour
Intermarche
NATIONAL
Carrefour
Intermarche
NATIONAL
Carrefour
Intermarche


### Slide 12 : Category Evolution Analysis 

In [90]:
manuf_brand_list = client_manuf + client_brands

In [91]:
if sectors:
    categoryEvolution(prs,cea_sectors_clients_new,globals()[f"cea_sectors_{slides_Period}_new"],cea_dya_sectors_new,cea_sectors_clients_total_new,manuf_brand_list,col="Sector",position = calculate_position(p))
    p+=1

In [92]:
if len(segments)!=0:
    posadd=calculate_position(p)
    categoryEvolution(prs,cea_segment_clients_new,globals()[f"cea_segment_{slides_Period}_new"],cea_dya_segment_new,cea_segment_clients_total_new,manuf_brand_list,col='Segment',position = posadd)
    p+=1    

In [93]:
if len(subsegments)!=0:
    posadd=calculate_position(p)
    categoryEvolution(prs,cea_subsegment_clients_new,globals()[f"cea_subsegment_{slides_Period}_new"],cea_dya_subsegment_new,cea_subsegment_clients_total_new,manuf_brand_list,col='SubSegment',position = posadd)
    p+=1


In [94]:
if len(subcategories)!=0:
    posadd=calculate_position(p)
    categoryEvolution(prs,cea_subcategory_clients_new,globals()[f"cea_subcategory_{slides_Period}_new"],cea_dya_subcategory_new,cea_subcategory_clients_total_new,manuf_brand_list,col='SubCategory',position = posadd)   
    p+=1  

### Slide 13 : Share And Growth

In [95]:
catSecSeg = (
    categories + sectors + segments + subsegments + subcategories
    if any([categories, sectors, segments, subsegments, subcategories])
    else []
)

###### Retailer

In [96]:
if not any([
    globals().get("modified_retailer_Region", False),
    globals().get("modified_retailer_Channel", False),
    globals().get("modified_retailer_Market", False)
]):
    # Skip the block
    pass
else:
    for key,val in retailerField.items():
        if val[0]:
 
            if categories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in categories}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
            if sectors:  
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerSecDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in sectors}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
 
            if segments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in segments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
 
            if subsegments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerSubSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in subsegments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
           
            if subcategories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerSubCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in subcategories}
 
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1

###### Channel

In [97]:
if not any([
    globals().get("modified_channels_Region", False),
    globals().get("modified_channels_Channel", False),
    globals().get("modified_channels_Market", False)
]):
    # Skip the block
    pass
else:
    for key,val in channelField.items():
        if val[0]:
            if categories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in channelCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in categories}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
           
 
            if sectors:  
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in channelSecDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in sectors}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
            if segments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in channelSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in segments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
 
            if subsegments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in channelSubSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in subsegments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
            if subcategories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in channelSubCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in subcategories}
 
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1

###### Region

In [98]:
if not any([
    globals().get("modified_regions_Region", False),
    globals().get("modified_regions_Channel", False),
    globals().get("modified_regions_Market", False)
]):
    # Skip the block
    pass
else:
    for key,val in regionField.items():
        if val[0]:
 
            if categories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in regionCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in categories}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
            if sectors:  
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in regionSecDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in sectors}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
 
            if segments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in regionSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in segments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
 
            if subsegments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in regionSubSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in subsegments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
           
            if subcategories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in regionSubCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in subcategories}
 
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1

###### Custom

In [99]:
if not any([
    globals().get("modified_cust_Region", False),
    globals().get("modified_cust_Channel", False),
    globals().get("modified_cust_Market", False)
]):
    # Skip the block
    pass
else:
    for key,val in custField.items():
        if val[0]:
 
            if categories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1] if key in custCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0] if key in categories}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
            if sectors:  
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1] if key in custSecDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0] if key in sectors}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
 
            if segments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1] if key in custSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0] if key in segments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
            if subsegments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1] if key in custSubSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0] if key in subsegments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
           
            if subcategories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1] if key in custSubCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0] if key in subcategories}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1

#### Momentum Analysis

In [100]:

if retailers:
    Momentum_Analysis(prs, list(modified_momentum_retailers.keys()), modified_momentum_retailers, position = calculate_position(p)) 
    p+=1

if channels:
    Momentum_Analysis(prs, list(modified_momentum_channels.keys()), modified_momentum_channels, position = calculate_position(p)) 
    p+=1

if regions:
    Momentum_Analysis(prs, list(modified_momentum_regions.keys()), modified_momentum_regions, position = calculate_position(p)) 
    p+=1


if cust:
    Momentum_Analysis(prs, list(modified_momentum_cust.keys()), modified_momentum_cust, position = calculate_position(p)) 
    p+=1


if segments:
    if retailers:
        Momentum_Analysis(prs, list(modified_momentum_retailers_seg.keys()), modified_momentum_retailers_seg, position = calculate_position(p)) 
        p+=1

    if channels:
        Momentum_Analysis(prs, list(modified_momentum_channels_seg.keys()), modified_momentum_channels_seg, position = calculate_position(p)) 
        p+=1

    if regions:
        Momentum_Analysis(prs, list(modified_momentum_regions_seg.keys()), modified_momentum_regions_seg, position = calculate_position(p)) 
        p+=1


    if cust:
        Momentum_Analysis(prs, list(modified_momentum_cust_seg.keys()), modified_momentum_cust_seg, position = calculate_position(p)) 
        p+=1

if subsegments:
    if retailers:
        Momentum_Analysis(prs, list(modified_momentum_retailers_subseg.keys()), modified_momentum_retailers_subseg, position = calculate_position(p)) 
        p+=1

    if channels:
        Momentum_Analysis(prs, list(modified_momentum_channels_subseg.keys()), modified_momentum_channels_subseg, position = calculate_position(p)) 
        p+=1

    if regions:
        Momentum_Analysis(prs, list(modified_momentum_regions_subseg.keys()), modified_momentum_regions_subseg, position = calculate_position(p)) 
        p+=1


    if cust:
        Momentum_Analysis(prs, list(modified_momentum_cust_subseg.keys()), modified_momentum_cust_subseg, position = calculate_position(p)) 
        p+=1

if subcategories:
    if retailers:
        Momentum_Analysis(prs, list(modified_momentum_retailers_subcatg.keys()), modified_momentum_retailers_subcatg, position = calculate_position(p)) 
        p+=1

    if channels:
        Momentum_Analysis(prs, list(modified_momentum_channels_subcatg.keys()), modified_momentum_channels_subcatg, position = calculate_position(p)) 
        p+=1

    if regions:
        Momentum_Analysis(prs, list(modified_momentum_regions_subcatg.keys()), modified_momentum_regions_subcatg, position = calculate_position(p)) 
        p+=1

    if cust:
        Momentum_Analysis(prs, list(modified_momentum_cust_subcatg.keys()), modified_momentum_cust_subcatg, position = calculate_position(p)) 
        p+=1


Momentum_Analysis(prs, list(modified_momentum_dfs.keys()), modified_momentum_dfs, position = calculate_position(p))

Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Total Fromage | RETAILER | Region
Soft Cheese | RETAILER | Region
Soft Cheese | RETAILER | Region
Soft Cheese | RETAILER | Region
Soft Cheese | RETAIL

In [101]:
outputPath = os.getcwd()+f"\\Landscape Pre-Output ShareGrowth_{datetime.today().strftime("%d-%m")}.pptx"
prs.save(outputPath)

In [102]:
output_path_final = os.getcwd()+f"\\Final Output\\Landscape ShareGrowth_{datetime.today().strftime("%d-%m")}.pptx"
open_chart_data_in_excel(outputPath,output_path_final)

Slide 94: Opened Excel workbook: Book1
Slide 95: Opened Excel workbook: Book1
Slide 96: Opened Excel workbook: Book1
Slide 97: Opened Excel workbook: Book1
Slide 98: Opened Excel workbook: Book1
Slide 99: Opened Excel workbook: Book1
Slide 100: Opened Excel workbook: Book1
Slide 101: Opened Excel workbook: Book1
Slide 102: Opened Excel workbook: Book1
Slide 103: Opened Excel workbook: Book1
Slide 104: Opened Excel workbook: Book1


# RPVM

In [103]:
variables = [
    "revenue_PVM_manuf",
    "revenue_PVM_brand"
]

for var in variables:
    if var in globals():
        globals()[var] = add_totals_after_grouping(globals()[var])


In [104]:
def RPVMClean(dic,rpvm_comp_brands,scope,MaunfORBrand,clientlis=[]):
   modified_revenue_PVM_Brand = {}
   for key in dic.keys():
      df = dic[key].copy()
      if key.split(" | ")[0] in scope:  
         df['Value Share'] = df['Value Share'].astype(float)
         df['Revenue By PVM'] = df['Revenue By PVM'].astype(float) / 1000
         relevant_client_brands = [brand for brand in clientlis if brand in df[f'{MaunfORBrand}'].values]
         if relevant_client_brands:
            if not rpvm_comp_brands:
               filtered_df = df[
                  (df[f"{MaunfORBrand}"].str.contains("Total")) & 
                  (df[f"{MaunfORBrand}"] != "Grand Total") &
                  (df[f"{MaunfORBrand}"] != "All Others Total") &  
                  (~df[f"{MaunfORBrand}"].str.replace(" Total", "").isin(clientlis))
               ].nlargest(1, "Value Share")
               top_competitors = filtered_df[f'{MaunfORBrand}'].str.replace(" Total", "").tolist()

            else:
               top_competitors = rpvm_comp_brands   #defined list
            if top_competitors:
               df[f"{MaunfORBrand}"] = df[f"{MaunfORBrand}"].str.strip()
               for brand in relevant_client_brands:
                     for comp in top_competitors:
                        filtered_df = df[(df[f'{MaunfORBrand}'] == brand) | (df[f'{MaunfORBrand}'] == comp)]
                        new_key = f"{key} | {brand} | {comp}"
                        if not filtered_df.empty:
                           modified_revenue_PVM_Brand[new_key] = filtered_df
   return  modified_revenue_PVM_Brand                 

In [105]:
if len(categories)!=0:
    modified_revenue_PVM_Brand=RPVMClean(revenue_PVM_brand,rpvm_comp_brands,categories,f'{BrandOrTopB}',client_brands)
    modified_revenue_PVM_Manuf=RPVMClean(revenue_PVM_manuf,rpvm_comp_brands,categories,f'{ManufOrTopC}',client_manuf)

if len(sectors)!=0:
    modified_revenue_PVM_Brand=RPVMClean(revenue_PVM_brand,rpvm_comp_brands,sectors,f'{BrandOrTopB}',client_brands)
    modified_revenue_PVM_Manuf=RPVMClean(revenue_PVM_manuf,rpvm_comp_brands,sectors,f'{ManufOrTopC}',client_manuf)
    

In [106]:
index = [*[14]*(1)]
index=[i for i in index if i !=[]]

duplication_1 = [len(modified_revenue_PVM_Brand.keys())]
# duplication_2=[len(modified_revenue_PVM_Brand.keys())]

duplication = duplication_1

duplication = [item for item in duplication if item !=0]

section_names_slide1 = ["Revenue PVM By Brands"]
# section_names_slide2 = ["Revenue PVM By Brands"]

section_names = [ *section_names_slide1]

In [107]:
path = os.getcwd() + '\Landscape base.pptx'
new_pre = os.getcwd() + '\Landscape Duplicate RPVM.pptx'

In [108]:
print(len(index))
print(index)
print(len(duplication))
print(duplication)
print(len(section_names))
print(section_names)
print(sum(duplication))

1
[14]
1
[18]
1
['Revenue PVM By Brands']
18


In [109]:
slideDuplication(index,duplication,section_names,path,new_pre)

com_error: (-2147221008, 'CoInitialize has not been called.', None, None)

In [ ]:
def calculate_position(end):
    return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))

ppt_app = win32com.client.Dispatch("PowerPoint.Application")

prs = ppt_app.Presentations.open(new_pre)

In [ ]:
combined_list_duplicates = list(modified_revenue_PVM_Brand.keys())
combined_modified_RPVM = { **modified_revenue_PVM_Brand}

RPVM(prs, combined_list_duplicates, combined_modified_RPVM, position=calculate_position(0))

Error on slide 0, retry 1: (-2147352567, 'Exception occurred.', (0, 'Microsoft PowerPoint', "View.GotoSlide : Integer out of range. 2 is not in Index's valid range of 1 to 1.", '', 0, -2147188160), None)
Error updating chart data on slide 0: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147467259), None)
Error on slide 0, retry 2: (-2147352567, 'Exception occurred.', (0, 'Microsoft PowerPoint', "View.GotoSlide : Integer out of range. 2 is not in Index's valid range of 1 to 1.", '', 0, -2147188160), None)
Error updating chart data on slide 0: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147467259), None)
Error updating chart data on slide 0: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147467259), None)
Error on slide 0, retry 3: (-2147023174, 'The RPC server is unavailable.', None, None)
Error updating chart data on slide 0: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147467259), None)
Error on slide 0, retry 4

In [ ]:

output_path = os.path.join(os.getcwd(), 'Landscape RPVM Trying.pptx')
prs.SaveAs(output_path)


# Slides WITHOUT forcing CLIENT BRAND

In [51]:
Sections_WO_CB=True

In [52]:
if not Sections_WO_CB:
    print("Stopping the script as 'Sections_WO_CB' is set to False.")
    sys.exit()  # Terminate the script here


In [53]:
modified_brands_share_noCB = {}
modified_brands_totals_noCB = {}
globals()[f"modified_brands_{slides_Period}_noCB"] = {}


for s in brands_dfs_new.keys():
    t = brands_dfs_new[s].copy()
    # t=DetectHeader(t).fillna(0)
    t.columns = t.columns.str.strip()
   
    t = t.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
    t = t.applymap(lambda x: 0 if pd.isna(x) else x)
    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    if not mod.empty:
        modified_brands_share_noCB[s] = mod
    if not tot.empty:
        modified_brands_totals_noCB[s] = tot


df_toDrop =[key for key in modified_brands_share_noCB.keys() if (modified_brands_share_noCB[key].shape[1]>2) and (modified_brands_share_noCB[key].shape[1]<4)]
for key in df_toDrop:
    del modified_brands_share_noCB[key]

    
emptyDf=[key for key in modified_brands_share_noCB.keys() if modified_brands_share_noCB[key].shape[1]<4]
for dic in [modified_brands_share_noCB]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            print()
            dic[df]=dic[df].drop(columns=[''])#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0
p = 10 if len(client_brands) < 10 else len(client_brands) + 1

modified_brands_share_noCB = dfSort(modified_brands_share_noCB, [],f'{BrandOrTopB}',p, 'Total Value Share')
for key in modified_brands_share_noCB.keys():
    modified_brands_share_noCB[key] = modified_brands_share_noCB[key][modified_brands_share_noCB[key]['Total Value Share'] != 0]
addOther(modified_brands_share_noCB,col=f'{BrandOrTopB}',TorB="Top")

brand_per=globals()[f"brand_{slides_Period}_dfs"]
for s in brand_per.keys():
    t = brand_per[s].copy()
    t.columns = t.columns.str.strip()
    t = t.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
    t = t.applymap(lambda x: 0 if pd.isna(x) else x)
    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    globals()[f"modified_brands_{slides_Period}_noCB"][s] = mod



df_toDrop =[key for key in globals()[f"modified_brands_{slides_Period}_noCB"].keys() if (globals()[f"modified_brands_{slides_Period}_noCB"][key].shape[1]>2) and (globals()[f"modified_brands_{slides_Period}_noCB"][key].shape[1]<4)]
for key in df_toDrop:
    del globals()[f"modified_brands_{slides_Period}_noCB"][key]

emptyDf=[key for key in globals()[f"modified_brands_{slides_Period}_noCB"].keys() if globals()[f"modified_brands_{slides_Period}_noCB"][key].shape[1]<4]
for dic in [globals()[f"modified_brands_{slides_Period}_noCB"]]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            dic[df]=dic[df].drop(columns=[''])#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0

globals()[f"modified_brands_{slides_Period}_noCB"] = dfSort(globals()[f"modified_brands_{slides_Period}_noCB"], [],f'{BrandOrTopB}',10, 'Value Share')


In [54]:
index = [
    *[0]*(1),
           *[1]*(1),
                4,
                sectorIndex if sectors else [],
                segmentIndex if segments else [],
                subsegmentIndex if subsegments else [],
                subcategoryIndex if subcategories else []
                ]
index=[i for i in index if i !=[]]

duplication_1 = [len(modified_brands_share_noCB.keys())]
duplication_2 = [len(modified_brands_share_noCB.keys())]

duplication_5 = [len(globals()[f"modified_manuf_{slides_Period}_new"].keys())] 
duplication_6 = [1 if sectors else 0, 1 if segments else 0,  1 if subsegments else 0,  1 if subcategories else 0]

duplication = duplication_1 + duplication_2  + duplication_5 + duplication_6


duplication = [item for item in duplication if item !=0]

section_names_slide1 = ["Market Trends by Brands"]
section_names_slide2 = ["Market Concentration By Brands"]
section_names_slide5 = ['Share and Growth By Manufacturer/Brands']
section_names_slide6 = ([ 'Share And Growth By Brands By Sector'] if len(sectors)>0 else [] ) + (['Share And Growth By Brands By Segment'] if len(segments)>0 else [] ) + (['Share And Growth By Brands By SubSegment'] if len(subsegments)>0 else [] )  + (['Share And Growth By Brands By SubCategory'] if len(subcategories)>0 else [] )

section_names = [ *section_names_slide1,*section_names_slide2,
                  *section_names_slide5, *section_names_slide6  
                 ]


In [55]:
path = os.getcwd() + '\Landscape base.pptx'
new_pre = os.getcwd() + '\Landscape Duplicate Without CB.pptx'

In [56]:
print(len(index))
# print(index)
print(len(duplication))
print(duplication)
print(len(section_names))
# print(section_names)
print(sum(duplication))

5
5
[21, 21, 21, 1, 1]
5
65


In [57]:
slideDuplication(index,duplication,section_names,path,new_pre)


In [58]:
def calculate_position(end):
    return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))

In [59]:
prs = Presentation(new_pre)


In [60]:
p = 0
Market_Trends(prs, list(modified_brands_share_noCB.keys()), modified_brands_share_noCB, modified_brands_totals_noCB, client_brands ,position =calculate_position(p), slide_by = f'{BrandOrTopB}', years=years)

p+=1

In [61]:
Market_Concentration(prs,list(modified_brands_share_noCB.keys()), modified_brands_share_noCB, modified_brands_totals_noCB, client_brands ,position =  calculate_position(p), slide_by = f'{BrandOrTopB}',years=years)

p+=1

In [62]:
for k in globals()[f"modified_brands_{slides_Period}_noCB"].keys():
    globals()[f"modified_brands_{slides_Period}_noCB"][k] = globals()[f"modified_brands_{slides_Period}_noCB"][k].rename(columns = { f"Value Share {slides_Period}":"Value Share", f"Share DYA {slides_Period}":"Share DYA", f"Av Price/KG {slides_Period}":"Av Price/KG"})
    globals()[f"modified_brands_{slides_Period}_noCB"][k] = globals()[f"modified_brands_{slides_Period}_noCB"][k].sort_values('Value Share', ascending = False)
    globals()[f"modified_brands_{slides_Period}_noCB"][k] = globals()[f"modified_brands_{slides_Period}_noCB"][k][globals()[f"modified_brands_{slides_Period}_noCB"][k]['Value Share']!=0]    
    globals()[f"modified_brands_{slides_Period}_noCB"][k]['Sorting'] = np.where(globals()[f"modified_brands_{slides_Period}_noCB"][k][f'{BrandOrTopB}'] == 'Others', 1, 0)
    globals()[f"modified_brands_{slides_Period}_noCB"][k] = globals()[f"modified_brands_{slides_Period}_noCB"][k].sort_values(['Sorting',"Value Share"], ascending = [True, False]).drop(columns = 'Sorting')


In [63]:
Share_Growth(prs, list(globals()[f"modified_manuf_{slides_Period}_new"].keys()), globals()[f"modified_manuf_{slides_Period}_new"], globals()[f"modified_brands_{slides_Period}_noCB"], position =calculate_position(p))

p+=1

In [64]:
for k in globals()[f"modified_brands_{slides_Period}_noCB"].keys():
    t=globals()[f"modified_brands_{slides_Period}_noCB"][k].copy()
    t= t.rename(columns = {"Value Share": f"Value Share {slides_Period}", "Share DYA":f"Share DYA {slides_Period}", "Av Price/KG":f"Av Price/KG {slides_Period}"})
    globals()[f"modified_brands_{slides_Period}_noCB"][k] = t[t[f'Value Share {slides_Period}'] != 0]

In [65]:
c = calculate_position(p)
if len(sectors)!=0:

    c=calculate_position(p)
    for key in sharGrowthDf_sec.keys():
        sectors_list = [sharGrowthDf_sec[key][i:i+3] for i in range(0, len(sharGrowthDf_sec[key]), 3)]
        for sublist in sectors_list:
            Share_Growth_Separately( prs,[key], globals()[f"modified_brands_{slides_Period}_noCB"],position=c,slide_by='Brands | By Sector', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  
    p+=1
if len(segments)!=0:
    
    c=calculate_position(p)
    for key in sharGrowthDf_seg.keys():
        segments_list = [sharGrowthDf_seg[key][i:i+3] for i in range(0, len(sharGrowthDf_seg[key]), 3)]
        for sublist in segments_list:
            Share_Growth_Separately( prs,[key],globals()[f"modified_brands_{slides_Period}_noCB"],position=c,slide_by='Brands | By Segment', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1

if len(subsegments)!=0:
    
    c=calculate_position(p)
    for key in sharGrowthDf_subseg.keys():
        subsegments_list = [sharGrowthDf_subseg[key][i:i+3] for i in range(0, len(sharGrowthDf_subseg[key]), 3)]
        for sublist in subsegments_list:
            Share_Growth_Separately( prs,[key], globals()[f"modified_brands_{slides_Period}_noCB"],position=c,slide_by='Brands | By SubSegment', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1


if len(subcategories)!=0:
    
    c=calculate_position(p)
    for key in sharGrowthDf_subcat.keys():
        subcateg_list = [sharGrowthDf_subcat[key][i:i+3] for i in range(0, len(sharGrowthDf_subcat[key]), 3)]
        for sublist in subcateg_list:
            Share_Growth_Separately( prs,[key], globals()[f"modified_brands_{slides_Period}_noCB"],position=c,slide_by='Brands | By SubCategory', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1        

NATIONAL
Carrefour
Intermarche
NATIONAL
Carrefour
Intermarche


In [66]:
outputPath = os.getcwd()+f"\\Final Output\\Landscape Output WO CB_{datetime.today().strftime("%d-%m")}.pptx"
prs.save(outputPath)

In [67]:
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Script started at: {time.ctime(start_time)}")
print(f"Script ended at: {time.ctime(end_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Script started at: Thu Sep 11 08:51:46 2025
Script ended at: Thu Sep 11 08:52:11 2025
Elapsed time: 25.64 seconds
